In [16]:
import json
import re
from pathlib import Path
from typing import Dict, Any, List, Tuple, Optional
from dotenv import load_dotenv
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import os
load_dotenv('chat.env')


True

In [17]:
EDIT_START = "<|editable_region_start|>"
EDIT_END = "<|editable_region_end|>"


In [18]:
def extract_region_from_input(model_input_text: str) -> str:
    """
    Extracts the code between <|editable_region_start|> and <|editable_region_end|>
    from model_input_text. Returns the region as a string (without the markers).
    Raises ValueError if markers are missing / misordered.
    """
    lines = model_input_text.split("\n")
    start_idx = None
    end_idx = None

    for i, line in enumerate(lines):
        if EDIT_START in line:
            start_idx = i
        if EDIT_END in line:
            end_idx = i
            break

    if start_idx is None or end_idx is None or end_idx <= start_idx:
        raise ValueError("Could not find valid editable region markers in model_input_text")

    # Region is between the markers (exclude lines that contain the markers)
    region_lines = []
    for i in range(start_idx + 1, end_idx):
        region_lines.append(lines[i])

    return "\n".join(region_lines).strip()


In [19]:
TOKEN_SPLIT_RE = re.compile(r"\W+")

def tokenize_for_jaccard(text: str) -> List[str]:
    tokens = TOKEN_SPLIT_RE.split(text)
    tokens = [t for t in tokens if t]  # drop empty
    return tokens


In [20]:
def jaccard_similarity(a: str, b: str) -> float:
    tokens_a = set(tokenize_for_jaccard(a))
    tokens_b = set(tokenize_for_jaccard(b))
    if not tokens_a and not tokens_b:
        return 1.0
    if not tokens_a or not tokens_b:
        return 0.0
    inter = len(tokens_a & tokens_b)
    union = len(tokens_a | tokens_b)
    return inter / union


In [21]:
def cosine_similarity(vec_a: np.ndarray, vec_b: np.ndarray) -> float:
    denom = (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))
    if denom == 0:
        return 0.0
    return float(np.dot(vec_a, vec_b) / denom)


In [22]:
class CodeEmbeddingModel:
    """
    Thin wrapper so you can swap models easily.

    Example models:
    - "jinaai/jina-embeddings-v2-base-code"
    - "nomic-ai/nomic-embed-text-v1.5"
    - "sentence-transformers/all-MiniLM-L6-v2" (for quick tests)
    """
    def __init__(self, model_name: str = "jinaai/jina-embeddings-v2-base-code"):
        self.model = SentenceTransformer(model_name)

    def encode(self, text: str) -> np.ndarray:
        return self.model.encode(text, normalize_embeddings=True)


In [34]:
import json
from openai import OpenAI

client = OpenAI()  # you already have this

def judge_pair_with_llm(
    client: OpenAI,
    model_input_text: str,
    model_target_text: str,
    language: str = "java",
    file_path: str = None,
    model: str = "gpt-4.1-mini",
) -> dict:
    """
    Uses Chat Completions API as a judge.
    Returns a dict like:
    {
      "label": "GOOD" | "GOOD_BUT_NOISY" | "BAD",
      "confidence": 1-5,
      "flags": [...],
      "reasons": "..."
    }
    """
    system_msg = (
        "You are a senior code-reviewer and dataset curator for a code completion model.\n"
        "You are given:\n"
        "1. A file's code with special markers <|editable_region_start|> and <|editable_region_end|> "
        "indicating an editable region.\n"
        "2. A proposed 'target' code snippet that is supposed to represent the final contents of that region in "
        "the edited version of the file.\n\n"
        "Your job is to decide whether this (input, target) pair is suitable training data for a code completion "
        "or fill-in-the-middle model.\n\n"
        "Consider:\n"
        "- Local syntax correctness and consistency with the language.\n"
        "- Whether the target snippet could reasonably be predicted from the input context.\n"
        "- Whether the target looks truncated, misaligned, or inconsistent with the region boundaries.\n"
        "- Whether the target appears unrelated to the input (e.g. totally different code).\n\n"
        "Respond ONLY in strict JSON with keys:\n"
        "- label: one of 'GOOD', 'GOOD_BUT_NOISY', 'BAD'\n"
        "- confidence: integer 1-5\n"
        "- reasons: short natural-language explanation\n"
        "- flags: list of strings like ['TRUNCATED', 'MISMATCHED_REGION', 'UNRELATED', 'MINOR_NOISE']\n"
    )

    user_payload = {
        "language": language,
        "file_path": file_path,
        "model_input_text": model_input_text,
        "model_target_text": model_target_text,
    }

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": json.dumps(user_payload)},
        ],
        temperature=0,
    )

    content = resp.choices[0].message.content

    # Try to parse JSON; if it fails, mark as BAD
    try:
        result = json.loads(content)
    except json.JSONDecodeError:
        result = {
            "label": "BAD",
            "confidence": 1,
            "reasons": f"Failed to parse model JSON: {content[:200]}",
            "flags": ["PARSER_ERROR"],
        }

    return result


In [35]:
import json
from typing import List, Dict, Any

def load_dataset(path: str) -> List[Dict[str, Any]]:
    """
    Robust loader:
    - If file is a single JSON array or object -> parse with json.load
    - If that fails, fall back to JSONL (one JSON per line)
    """
    with open(path, "r", encoding="utf-8") as f:
        text = f.read().strip()

    # Try: full JSON first
    try:
        obj = json.loads(text)
        if isinstance(obj, list):
            # File is a JSON array of records
            return obj
        elif isinstance(obj, dict):
            # Maybe wrapped as {"records": [...]}
            if "records" in obj and isinstance(obj["records"], list):
                return obj["records"]
            # Otherwise just wrap in a list
            return [obj]
    except json.JSONDecodeError:
        pass  # not a single valid JSON document → try JSONL

    # Fallback: treat as JSON Lines
    records: List[Dict[str, Any]] = []
    with open(path, "r", encoding="utf-8") as f:
        for ln, line in enumerate(f, start=1):
            line = line.strip()
            if not line:
                continue
            try:
                rec = json.loads(line)
            except json.JSONDecodeError as e:
                print(f"[WARN] Skipping line {ln}: JSON decode error: {e}")
                continue
            records.append(rec)

    return records



In [39]:
def evaluate_record(
    record: Dict[str, Any],
    embedder: CodeEmbeddingModel,
    client: Optional[OpenAI] = None,
    use_llm: bool = True,
) -> Dict[str, Any]:
    """
    Compute:
    - region_before (from model_input_text)
    - Jaccard similarity
    - embedding cosine similarity
    - optional LLM judgement

    Returns an enriched dict with metrics + llm_judge.
    """
    model_input_text = record.get("model_input_text", "")
    model_target_text = record.get("model_target_text", "")
    language = record.get("file_extension", "java")
    file_path = record.get("file_path")

    # 1) Validate model_target_text
    if not isinstance(model_target_text, str) or model_target_text.strip() == "":
        # No valid target → can't be used for training
        record.update(
            {
                "region_before": "",
                "jaccard_similarity": 0.0,
                "semantic_cosine_similarity": 0.0,
                "llm_judge": {
                    "label": "BAD",
                    "confidence": 5,
                    "reasons": "model_target_text is missing or not a string",
                    "flags": ["INVALID_TARGET"],
                },
            }
        )
        return record

    # 2) Extract region_before from model_input_text
    try:
        region_before = extract_region_from_input(model_input_text)
    except ValueError as e:
        # No valid region; mark as definitely bad
        record.update(
            {
                "region_before": "",
                "jaccard_similarity": 0.0,
                "semantic_cosine_similarity": 0.0,
                "llm_judge": {
                    "label": "BAD",
                    "confidence": 5,
                    "reasons": f"Invalid editable region markers: {e}",
                    "flags": ["INVALID_REGION"],
                },
            }
        )
        return record

    # 3) Jaccard similarity between before-region and target
    jac = jaccard_similarity(region_before, model_target_text)

    # 4) Semantic similarity between before-region and target
    emb_before = embedder.encode(region_before)
    emb_target = embedder.encode(model_target_text)
    sem_cos = cosine_similarity(emb_before, emb_target)

    # 5) LLM-as-judge (optional)
    llm_result = None
    if use_llm and client is not None:
        llm_result = judge_pair_with_llm(
            client=client,
            model_input_text=model_input_text,
            model_target_text=model_target_text,
            language=language,
            file_path=file_path,
        )

    record.update(
        {
            "region_before": region_before,
            "jaccard_similarity": jac,
            "semantic_cosine_similarity": sem_cos,
            "llm_judge": llm_result,
        }
    )
    return record


In [37]:
input_path = "/mnt/new-ssd/extra-space/sharvay/GitData/datasets/import_modifications_only_dataset.json"
output_path = "/mnt/new-ssd/extra-space/sharvay/GitData/datasets/dataset_evaluated.jsonl"

records = load_dataset(input_path)
print("Loaded records:", len(records))


Loaded records: 2484


In [40]:
embedder = CodeEmbeddingModel(
    model_name="jinaai/jina-embeddings-v2-base-code"
)

evaluated = []
for rec in tqdm(records, desc="Evaluating records"):
    rec_eval = evaluate_record(rec, embedder=embedder, client=client, use_llm=True)
    evaluated.append(rec_eval)

save_jsonl(evaluated, output_path)


Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-code and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.10.output.dense.bias', 'encoder.layer.10.output.dense.weight', 'encoder.layer.11.intermediate.dense.bias', 'encoder.layer.11.interme

In [ ]:
{"commit_sha": "0442ba8c76b91a6afead3996e4702d677916717f", "commit_short_sha": "0442ba8c", "commit_message": "update playground example", "commit_author": "chickenlj", "commit_date": "2024-09-18T13:45:02+08:00", "total_files_in_commit": 6, "sequence_order": 1, "file_path": "spring-ai-alibaba-examples/playground-flight-booking/frontend/generated/vaadin.ts", "file_extension": "ts", "change_type": "modified", "lines_added": 26, "lines_deleted": 0, "total_changes": 26, "before_content": "import 'Frontend/generated/jar-resources/copilot/copilot.js';\n// @ts-ignore\nif (import.meta.hot) {\n  // @ts-ignore\n  import.meta.hot.on('vite:afterUpdate', () => {\n    const eventbus = (window as any).Vaadin.copilot.eventbus;\n    if (eventbus) {\n      eventbus.emit('vite-after-update',{});\n    }\n  });\n}\n\nimport '@vaadin/vertical-layout/theme/lumo/vaadin-vertical-layout.js';\nimport '@vaadin/horizontal-layout/theme/lumo/vaadin-horizontal-layout.js';\nimport '@vaadin/context-menu/theme/lumo/vaadin-context-menu.js';\nimport '@vaadin/checkbox/theme/lumo/vaadin-checkbox.js';\nimport '@vaadin/text-field/theme/lumo/vaadin-text-field.js';\nimport '@vaadin/text-area/theme/lumo/vaadin-text-area.js';\nimport '@vaadin/menu-bar/theme/lumo/vaadin-menu-bar.js';\nimport '@vaadin/grid/theme/lumo/vaadin-grid.js';\nimport '@vaadin/grid/theme/lumo/vaadin-grid-tree-column.js';\nimport '@vaadin/details/theme/lumo/vaadin-details.js';\nimport '@vaadin/select/theme/lumo/vaadin-select.js';\nimport '@vaadin/overlay/theme/lumo/vaadin-overlay.js';\nimport '@vaadin/list-box/theme/lumo/vaadin-list-box.js';\nimport '@vaadin/combo-box/theme/lumo/vaadin-combo-box.js';\nimport '@vaadin/item/theme/lumo/vaadin-item.js';\nimport '@vaadin/dialog/theme/lumo/vaadin-dialog.js';\nimport '@vaadin/multi-select-combo-box/theme/lumo/vaadin-multi-select-combo-box.js';\nimport '@vaadin/icons/vaadin-iconset.js';\nimport '@vaadin/icon/vaadin-icon.js';\nimport './vaadin-featureflags.js';\n\nimport './index';\n\nimport './vaadin-react.js';\nimport 'Frontend/generated/jar-resources/vaadin-dev-tools/vaadin-dev-tools.js';\n\nimport './theme-customer-support-agent.global.generated.js';\nimport { applyTheme } from './theme.js';\napplyTheme(document);\n", "after_content": "import 'Frontend/generated/jar-resources/copilot/copilot.js';\n// @ts-ignore\nif (import.meta.hot) {\n  // @ts-ignore\n  import.meta.hot.on('vite:afterUpdate', () => {\n    const eventbus = (window as any).Vaadin.copilot.eventbus;\n    if (eventbus) {\n      eventbus.emit('vite-after-update',{});\n    }\n  });\n}\n\nimport '@vaadin/vertical-layout/theme/lumo/vaadin-vertical-layout.js';\nimport '@vaadin/horizontal-layout/theme/lumo/vaadin-horizontal-layout.js';\nimport '@vaadin/context-menu/theme/lumo/vaadin-context-menu.js';\nimport '@vaadin/checkbox/theme/lumo/vaadin-checkbox.js';\nimport '@vaadin/text-field/theme/lumo/vaadin-text-field.js';\nimport '@vaadin/text-area/theme/lumo/vaadin-text-area.js';\nimport '@vaadin/menu-bar/theme/lumo/vaadin-menu-bar.js';\nimport '@vaadin/grid/theme/lumo/vaadin-grid.js';\nimport '@vaadin/grid/theme/lumo/vaadin-grid-tree-column.js';\nimport '@vaadin/details/theme/lumo/vaadin-details.js';\nimport '@vaadin/select/theme/lumo/vaadin-select.js';\nimport '@vaadin/overlay/theme/lumo/vaadin-overlay.js';\nimport '@vaadin/list-box/theme/lumo/vaadin-list-box.js';\nimport '@vaadin/combo-box/theme/lumo/vaadin-combo-box.js';\nimport '@vaadin/item/theme/lumo/vaadin-item.js';\nimport '@vaadin/dialog/theme/lumo/vaadin-dialog.js';\nimport '@vaadin/multi-select-combo-box/theme/lumo/vaadin-multi-select-combo-box.js';\nimport '@vaadin/icons/vaadin-iconset.js';\nimport '@vaadin/icon/vaadin-icon.js';\nimport client from 'Frontend/generated/connect-client.default.js';\nlet generatedId = 0;\nconst copilotMiddleware: any = async function(\n  context: any,\n  next: any\n) {\n    generatedId++;\n    const requestData = { endpoint: context.endpoint, method: context.method, params: context.params, id: generatedId };\n    (window as any).Vaadin.copilot.eventbus.emit('endpoint-request', requestData);\n\n    const response: Response = await next(context);\n    const text = await response.clone().text();\n    const responseData = { text, id: generatedId} ;\n    (window as any).Vaadin.copilot.eventbus.emit('endpoint-response', responseData);\n\n    return response;\n};\n\nclient.middlewares = [...client.middlewares, copilotMiddleware];\n\nimport './vaadin-featureflags.js';\n\nimport './index';\n\nimport './vaadin-react.js';\nimport 'Frontend/generated/jar-resources/vaadin-dev-tools/vaadin-dev-tools.js';\n\nimport './theme-customer-support-agent.global.generated.js';\nimport { applyTheme } from './theme.js';\napplyTheme(document);\n\nimport { Outlet } from 'react-router-dom';\n(window as any).Vaadin ??= {};\n(window as any).Vaadin.copilot ??= {};\n(window as any).Vaadin.copilot._ref ??= {};\n(window as any).Vaadin.copilot._ref.Outlet = Outlet;\n", "patch": "@@ -29,6 +29,26 @@ import '@vaadin/dialog/theme/lumo/vaadin-dialog.js';\n import '@vaadin/multi-select-combo-box/theme/lumo/vaadin-multi-select-combo-box.js';\n import '@vaadin/icons/vaadin-iconset.js';\n import '@vaadin/icon/vaadin-icon.js';\n+import client from 'Frontend/generated/connect-client.default.js';\n+let generatedId = 0;\n+const copilotMiddleware: any = async function(\n+  context: any,\n+  next: any\n+) {\n+    generatedId++;\n+    const requestData = { endpoint: context.endpoint, method: context.method, params: context.params, id: generatedId };\n+    (window as any).Vaadin.copilot.eventbus.emit('endpoint-request', requestData);\n+\n+    const response: Response = await next(context);\n+    const text = await response.clone().text();\n+    const responseData = { text, id: generatedId} ;\n+    (window as any).Vaadin.copilot.eventbus.emit('endpoint-response', responseData);\n+\n+    return response;\n+};\n+\n+client.middlewares = [...client.middlewares, copilotMiddleware];\n+\n import './vaadin-featureflags.js';\n \n import './index';\n@@ -39,3 +59,9 @@ import 'Frontend/generated/jar-resources/vaadin-dev-tools/vaadin-dev-tools.js';\n import './theme-customer-support-agent.global.generated.js';\n import { applyTheme } from './theme.js';\n applyTheme(document);\n+\n+import { Outlet } from 'react-router-dom';\n+(window as any).Vaadin ??= {};\n+(window as any).Vaadin.copilot ??= {};\n+(window as any).Vaadin.copilot._ref ??= {};\n+(window as any).Vaadin.copilot._ref.Outlet = Outlet;\n", "editable_region_start": 28, "editable_region_end": 42, "change_labels": null, "chunk_id": 0, "total_chunks_in_file": 1, "is_multi_chunk_file": false, "model_input_text": "import 'Frontend/generated/jar-resources/copilot/copilot.js';\n// @ts-ignore\nif (import.meta.hot) {\n  // @ts-ignore\n  import.meta.hot.on('vite:afterUpdate', () => {\n    const eventbus = (window as any).Vaadin.copilot.eventbus;\n    if (eventbus) {\n      eventbus.emit('vite-after-update',{});\n    }\n  });\n}\n\nimport '@vaadin/vertical-layout/theme/lumo/vaadin-vertical-layout.js';\nimport '@vaadin/horizontal-layout/theme/lumo/vaadin-horizontal-layout.js';\nimport '@vaadin/context-menu/theme/lumo/vaadin-context-menu.js';\nimport '@vaadin/checkbox/theme/lumo/vaadin-checkbox.js';\nimport '@vaadin/text-field/theme/lumo/vaadin-text-field.js';\nimport '@vaadin/text-area/theme/lumo/vaadin-text-area.js';\nimport '@vaadin/menu-bar/theme/lumo/vaadin-menu-bar.js';\nimport '@vaadin/grid/theme/lumo/vaadin-grid.js';\nimport '@vaadin/grid/theme/lumo/vaadin-grid-tree-column.js';\nimport '@vaadin/details/theme/lumo/vaadin-details.js';\nimport '@vaadin/select/theme/lumo/vaadin-select.js';\nimport '@vaadin/overlay/theme/lumo/vaadin-overlay.js';\nimport '@vaadin/list-box/theme/lumo/vaadin-list-box.js';\nimport '@vaadin/combo-box/theme/lumo/vaadin-combo-box.js';\nimport '@vaadin/item/theme/lumo/vaadin-item.js';\n<|editable_region_start|>\nimport '@vaadin/dialog/theme/lumo/vaadin-dialog.js';\nimport '@vaadin/multi-select-combo-box/theme/lumo/vaadin-multi-select-combo-box.js';\nimport '@vaadin/icons/vaadin-iconset.js';\nimport '@vaadin/icon/vaadin-icon.js';\nimport './vaadin-featureflags.js';\n\nimport './index';\n\nimport './vaadin-react.js';\nimport 'Frontend/generated/jar-resources/vaadin-dev-tools/vaadin-dev-tools.js';\n\nimport './theme-customer-support-agent.global.generated.js';\nimport { applyTheme } from './theme.js';\napplyTheme(document);\n\n<|editable_region_end|>", "model_target_text": "import '@vaadin/dialog/theme/lumo/vaadin-dialog.js';\nimport '@vaadin/multi-select-combo-box/theme/lumo/vaadin-multi-select-combo-box.js';\nimport '@vaadin/icons/vaadin-iconset.js';\nimport '@vaadin/icon/vaadin-icon.js';\nimport client from 'Frontend/generated/connect-client.default.js';\nlet generatedId = 0;\nconst copilotMiddleware: any = async function(\n  context: any,\n  next: any\n) {\n    generatedId++;\n    const requestData = { endpoint: context.endpoint, method: context.method, params: context.params, id: generatedId };\n    (window as any).Vaadin.copilot.eventbus.emit('endpoint-request', requestData);\n\n    const response: Response = await next(context);", "region_before": "import '@vaadin/dialog/theme/lumo/vaadin-dialog.js';\nimport '@vaadin/multi-select-combo-box/theme/lumo/vaadin-multi-select-combo-box.js';\nimport '@vaadin/icons/vaadin-iconset.js';\nimport '@vaadin/icon/vaadin-icon.js';\nimport './vaadin-featureflags.js';\n\nimport './index';\n\nimport './vaadin-react.js';\nimport 'Frontend/generated/jar-resources/vaadin-dev-tools/vaadin-dev-tools.js';\n\nimport './theme-customer-support-agent.global.generated.js';\nimport { applyTheme } from './theme.js';\napplyTheme(document);", "jaccard_similarity": 0.2807017543859649, "semantic_cosine_similarity": 1.0000001192092896, "llm_judge": {"label": "BAD", "confidence": 5, "reasons": "The target snippet is truncated and incomplete, ending abruptly inside a function. It introduces new code (copilotMiddleware) unrelated to the original imports and theme application in the editable region. The target does not align with the region boundaries and is inconsistent with the input context.", "flags": ["TRUNCATED", "MISMATCHED_REGION", "UNRELATED"]}}
{"commit_sha": "09eed911a7e0afa37e0c8e6ba72dd40731cbf391", "commit_short_sha": "09eed911", "commit_message": "update example", "commit_author": "chickenlj", "commit_date": "2024-09-18T14:32:19+08:00", "total_files_in_commit": 4, "sequence_order": 2, "file_path": "spring-ai-alibaba-examples/playground-flight-booking/frontend/generated/vaadin.ts", "file_extension": "ts", "change_type": "modified", "lines_added": 0, "lines_deleted": 59, "total_changes": 59, "before_content": "import 'Frontend/generated/jar-resources/copilot/copilot.js';\n// @ts-ignore\nif (import.meta.hot) {\n  // @ts-ignore\n  import.meta.hot.on('vite:afterUpdate', () => {\n    const eventbus = (window as any).Vaadin.copilot.eventbus;\n    if (eventbus) {\n      eventbus.emit('vite-after-update',{});\n    }\n  });\n}\n\nimport '@vaadin/vertical-layout/theme/lumo/vaadin-vertical-layout.js';\nimport '@vaadin/horizontal-layout/theme/lumo/vaadin-horizontal-layout.js';\nimport '@vaadin/context-menu/theme/lumo/vaadin-context-menu.js';\nimport '@vaadin/checkbox/theme/lumo/vaadin-checkbox.js';\nimport '@vaadin/text-field/theme/lumo/vaadin-text-field.js';\nimport '@vaadin/text-area/theme/lumo/vaadin-text-area.js';\nimport '@vaadin/menu-bar/theme/lumo/vaadin-menu-bar.js';\nimport '@vaadin/grid/theme/lumo/vaadin-grid.js';\nimport '@vaadin/grid/theme/lumo/vaadin-grid-tree-column.js';\nimport '@vaadin/details/theme/lumo/vaadin-details.js';\nimport '@vaadin/select/theme/lumo/vaadin-select.js';\nimport '@vaadin/overlay/theme/lumo/vaadin-overlay.js';\nimport '@vaadin/list-box/theme/lumo/vaadin-list-box.js';\nimport '@vaadin/combo-box/theme/lumo/vaadin-combo-box.js';\nimport '@vaadin/item/theme/lumo/vaadin-item.js';\nimport '@vaadin/dialog/theme/lumo/vaadin-dialog.js';\nimport '@vaadin/multi-select-combo-box/theme/lumo/vaadin-multi-select-combo-box.js';\nimport '@vaadin/icons/vaadin-iconset.js';\nimport '@vaadin/icon/vaadin-icon.js';\nimport client from 'Frontend/generated/connect-client.default.js';\nlet generatedId = 0;\nconst copilotMiddleware: any = async function(\n  context: any,\n  next: any\n) {\n    generatedId++;\n    const requestData = { endpoint: context.endpoint, method: context.method, params: context.params, id: generatedId };\n    (window as any).Vaadin.copilot.eventbus.emit('endpoint-request', requestData);\n\n    const response: Response = await next(context);\n    const text = await response.clone().text();\n    const responseData = { text, id: generatedId} ;\n    (window as any).Vaadin.copilot.eventbus.emit('endpoint-response', responseData);\n\n    return response;\n};\n\nclient.middlewares = [...client.middlewares, copilotMiddleware];\n\nimport './vaadin-featureflags.js';\n\nimport './index';\n\nimport './vaadin-react.js';\nimport 'Frontend/generated/jar-resources/vaadin-dev-tools/vaadin-dev-tools.js';\n\nimport './theme-customer-support-agent.global.generated.js';\nimport { applyTheme } from './theme.js';\napplyTheme(document);\n\nimport { Outlet } from 'react-router-dom';\n(window as any).Vaadin ??= {};\n(window as any).Vaadin.copilot ??= {};\n(window as any).Vaadin.copilot._ref ??= {};\n(window as any).Vaadin.copilot._ref.Outlet = Outlet;\n", "after_content": "import './vaadin-featureflags.js';\n\nimport './index';\n\nimport './vaadin-react.js';\nimport './theme-customer-support-agent.global.generated.js';\nimport { applyTheme } from './theme.js';\napplyTheme(document);\n", "patch": "@@ -1,67 +1,8 @@\n-import 'Frontend/generated/jar-resources/copilot/copilot.js';\n-// @ts-ignore\n-if (import.meta.hot) {\n-  // @ts-ignore\n-  import.meta.hot.on('vite:afterUpdate', () => {\n-    const eventbus = (window as any).Vaadin.copilot.eventbus;\n-    if (eventbus) {\n-      eventbus.emit('vite-after-update',{});\n-    }\n-  });\n-}\n-\n-import '@vaadin/vertical-layout/theme/lumo/vaadin-vertical-layout.js';\n-import '@vaadin/horizontal-layout/theme/lumo/vaadin-horizontal-layout.js';\n-import '@vaadin/context-menu/theme/lumo/vaadin-context-menu.js';\n-import '@vaadin/checkbox/theme/lumo/vaadin-checkbox.js';\n-import '@vaadin/text-field/theme/lumo/vaadin-text-field.js';\n-import '@vaadin/text-area/theme/lumo/vaadin-text-area.js';\n-import '@vaadin/menu-bar/theme/lumo/vaadin-menu-bar.js';\n-import '@vaadin/grid/theme/lumo/vaadin-grid.js';\n-import '@vaadin/grid/theme/lumo/vaadin-grid-tree-column.js';\n-import '@vaadin/details/theme/lumo/vaadin-details.js';\n-import '@vaadin/select/theme/lumo/vaadin-select.js';\n-import '@vaadin/overlay/theme/lumo/vaadin-overlay.js';\n-import '@vaadin/list-box/theme/lumo/vaadin-list-box.js';\n-import '@vaadin/combo-box/theme/lumo/vaadin-combo-box.js';\n-import '@vaadin/item/theme/lumo/vaadin-item.js';\n-import '@vaadin/dialog/theme/lumo/vaadin-dialog.js';\n-import '@vaadin/multi-select-combo-box/theme/lumo/vaadin-multi-select-combo-box.js';\n-import '@vaadin/icons/vaadin-iconset.js';\n-import '@vaadin/icon/vaadin-icon.js';\n-import client from 'Frontend/generated/connect-client.default.js';\n-let generatedId = 0;\n-const copilotMiddleware: any = async function(\n-  context: any,\n-  next: any\n-) {\n-    generatedId++;\n-    const requestData = { endpoint: context.endpoint, method: context.method, params: context.params, id: generatedId };\n-    (window as any).Vaadin.copilot.eventbus.emit('endpoint-request', requestData);\n-\n-    const response: Response = await next(context);\n-    const text = await response.clone().text();\n-    const responseData = { text, id: generatedId} ;\n-    (window as any).Vaadin.copilot.eventbus.emit('endpoint-response', responseData);\n-\n-    return response;\n-};\n-\n-client.middlewares = [...client.middlewares, copilotMiddleware];\n-\n import './vaadin-featureflags.js';\n \n import './index';\n \n import './vaadin-react.js';\n-import 'Frontend/generated/jar-resources/vaadin-dev-tools/vaadin-dev-tools.js';\n-\n import './theme-customer-support-agent.global.generated.js';\n import { applyTheme } from './theme.js';\n applyTheme(document);\n-\n-import { Outlet } from 'react-router-dom';\n-(window as any).Vaadin ??= {};\n-(window as any).Vaadin.copilot ??= {};\n-(window as any).Vaadin.copilot._ref ??= {};\n-(window as any).Vaadin.copilot._ref.Outlet = Outlet;\n", "editable_region_start": 1, "editable_region_end": 68, "change_labels": null, "chunk_id": 0, "total_chunks_in_file": 1, "is_multi_chunk_file": false, "model_input_text": "<|editable_region_start|>\nimport 'Frontend/generated/jar-resources/copilot/copilot.js';\n// @ts-ignore\nif (import.meta.hot) {\n  // @ts-ignore\n  import.meta.hot.on('vite:afterUpdate', () => {\n    const eventbus = (window as any).Vaadin.copilot.eventbus;\n    if (eventbus) {\n      eventbus.emit('vite-after-update',{});\n    }\n  });\n}\n\nimport '@vaadin/vertical-layout/theme/lumo/vaadin-vertical-layout.js';\nimport '@vaadin/horizontal-layout/theme/lumo/vaadin-horizontal-layout.js';\nimport '@vaadin/context-menu/theme/lumo/vaadin-context-menu.js';\nimport '@vaadin/checkbox/theme/lumo/vaadin-checkbox.js';\nimport '@vaadin/text-field/theme/lumo/vaadin-text-field.js';\nimport '@vaadin/text-area/theme/lumo/vaadin-text-area.js';\nimport '@vaadin/menu-bar/theme/lumo/vaadin-menu-bar.js';\nimport '@vaadin/grid/theme/lumo/vaadin-grid.js';\nimport '@vaadin/grid/theme/lumo/vaadin-grid-tree-column.js';\nimport '@vaadin/details/theme/lumo/vaadin-details.js';\nimport '@vaadin/select/theme/lumo/vaadin-select.js';\nimport '@vaadin/overlay/theme/lumo/vaadin-overlay.js';\nimport '@vaadin/list-box/theme/lumo/vaadin-list-box.js';\nimport '@vaadin/combo-box/theme/lumo/vaadin-combo-box.js';\nimport '@vaadin/item/theme/lumo/vaadin-item.js';\nimport '@vaadin/dialog/theme/lumo/vaadin-dialog.js';\nimport '@vaadin/multi-select-combo-box/theme/lumo/vaadin-multi-select-combo-box.js';\nimport '@vaadin/icons/vaadin-iconset.js';\nimport '@vaadin/icon/vaadin-icon.js';\nimport client from 'Frontend/generated/connect-client.default.js';\nlet generatedId = 0;\nconst copilotMiddleware: any = async function(\n  context: any,\n  next: any\n) {\n    generatedId++;\n    const requestData = { endpoint: context.endpoint, method: context.method, params: context.params, id: generatedId };\n    (window as any).Vaadin.copilot.eventbus.emit('endpoint-request', requestData);\n\n    const response: Response = await next(context);\n    const text = await response.clone().text();\n    const responseData = { text, id: generatedId} ;\n    (window as any).Vaadin.copilot.eventbus.emit('endpoint-response', responseData);\n\n    return response;\n};\n\nclient.middlewares = [...client.middlewares, copilotMiddleware];\n\nimport './vaadin-featureflags.js';\n\nimport './index';\n\nimport './vaadin-react.js';\nimport 'Frontend/generated/jar-resources/vaadin-dev-tools/vaadin-dev-tools.js';\n\nimport './theme-customer-support-agent.global.generated.js';\nimport { applyTheme } from './theme.js';\napplyTheme(document);\n\nimport { Outlet } from 'react-router-dom';\n(window as any).Vaadin ??= {};\n(window as any).Vaadin.copilot ??= {};\n(window as any).Vaadin.copilot._ref ??= {};\n(window as any).Vaadin.copilot._ref.Outlet = Outlet;\n\n<|editable_region_end|>", "model_target_text": null, "region_before": "", "jaccard_similarity": 0.0, "semantic_cosine_similarity": 0.0, "llm_judge": {"label": "BAD", "confidence": 5, "reasons": "model_target_text is missing or not a string", "flags": ["INVALID_TARGET"]}}
{"commit_sha": "5840b7349a002a847c6c3070276e040b8381e0d0", "commit_short_sha": "5840b734", "commit_message": "fix: incrementalOutput not working (#17)\n\n1. set enable search to false for default chat configuration.\r\n2. Fix incremental output issue and support streaming function call.", "commit_author": "TianBing Ye", "commit_date": "2024-09-23T16:46:24+08:00", "total_files_in_commit": 8, "sequence_order": 7, "file_path": "spring-ai-alibaba-core/src/test/java/com/alibaba/cloud/ai/dashscope/chat/client/DashScopeChatClientIT.java", "file_extension": "java", "change_type": "modified", "lines_added": 27, "lines_deleted": 14, "total_changes": 41, "before_content": "/*\n * Copyright 2023-2024 the original author or authors.\n *\n * Licensed under the Apache License, Version 2.0 (the \"License\");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *      https://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an \"AS IS\" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\n\npackage com.alibaba.cloud.ai.dashscope.chat.client;\n\nimport com.alibaba.cloud.ai.advisor.DocumentRetrievalAdvisor;\nimport com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\nimport com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\nimport com.alibaba.cloud.ai.dashscope.embedding.DashScopeEmbeddingModel;\nimport com.alibaba.cloud.ai.dashscope.rag.*;\nimport org.junit.jupiter.api.Assertions;\nimport org.junit.jupiter.api.Test;\nimport org.slf4j.Logger;\nimport org.slf4j.LoggerFactory;\nimport org.springframework.ai.chat.client.ChatClient;\nimport org.springframework.ai.chat.client.advisor.MessageChatMemoryAdvisor;\nimport org.springframework.ai.chat.memory.InMemoryChatMemory;\nimport org.springframework.ai.chat.model.ChatResponse;\nimport org.springframework.ai.document.Document;\nimport org.springframework.ai.document.DocumentRetriever;\nimport org.springframework.beans.factory.annotation.Autowired;\nimport org.springframework.beans.factory.annotation.Value;\nimport org.springframework.boot.test.context.SpringBootTest;\nimport org.springframework.core.io.Resource;\nimport org.springframework.test.context.TestPropertySource;\nimport reactor.core.publisher.Flux;\n\nimport java.io.IOException;\nimport java.nio.charset.StandardCharsets;\nimport java.util.Arrays;\nimport java.util.HashMap;\nimport java.util.List;\nimport java.util.UUID;\nimport java.util.concurrent.CountDownLatch;\n\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_CONVERSATION_ID_KEY;\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_RETRIEVE_SIZE_KEY;\n\n/**\n * Title React agent test cases.<br/>\n * Description React agent test cases.<br/>\n * Created at 2024-08-19 10:15\n *\n * @author yuanci.ytb\n * @version 1.0.0\n * @since jdk8\n */\n\n@TestPropertySource(\"classpath:application.yml\")\n@SpringBootTest(classes = { DashScopeAutoConfiguration.class, DashScopeFunctionTestConfiguration.class,\n\t\tMockOrderService.class })\npublic class DashScopeChatClientIT {\n\n\tprivate static final Logger logger = LoggerFactory.getLogger(DashScopeChatClientIT.class);\n\n\t@Autowired\n\tprivate DashScopeChatModel dashscopeChatModel;\n\n\t@Autowired\n\tprivate DashScopeApi dashscopeApi;\n\n\t@Value(\"classpath:/prompts/rag/system-qa.st\")\n\tprivate Resource systemResource;\n\n\t@Value(\"classpath:/prompts/rag/system-qa-ref.st\")\n\tprivate Resource systemResourceRef;\n\n\t@Test\n\tvoid callTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"spring ai alibaba 是什么?\").call().chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithFunctionTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.function(\"getWeather\", \"根据城市查询天气\", new MockWeatherService())\n\t\t\t.user(\"杭州今天的天气如何?\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionBeanTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).defaultFunctions(\"getOrderFunction\").build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.functions(\"getCurrentWeather\")\n\t\t\t.user(\"帮我一下订单, 用户编号为1001, 订单编号为2001\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionAndRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"上海今天的天气如何?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithReferencedRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t\tList<Document> documents = (List<Document>) response.getMetadata()\n\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\tAssertions.assertNotNull(documents);\n\n\t\tfor (Document document : documents) {\n\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t}\n\t}\n\n\t@Test\n\tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\n\t\t\tChatCompletionFinishReason finishReason = ChatCompletionFinishReason\n\t\t\t\t.valueOf(data.getResult().getMetadata().getFinishReason());\n\t\t\tif (finishReason != ChatCompletionFinishReason.NULL) {\n\t\t\t\tList<Document> documents = (List<Document>) data.getMetadata()\n\t\t\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\t\t\tAssertions.assertNotNull(documents);\n\n\t\t\t\tfor (Document document : documents) {\n\t\t\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t\t\t}\n\t\t\t}\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithMemory() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultAdvisors(new MessageChatMemoryAdvisor(new InMemoryChatMemory()))\n\t\t\t.build();\n\n\t\tString conversantId = UUID.randomUUID().toString();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\n\t\tresponse = chatClient.prompt()\n\t\t\t.user(\"可以给一些使用示例吗?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tcontent = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid reader() {\n\t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeApi, null);\n\t\tList<Document> documentList = reader.get();\n\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeApi);\n\t\tList<Document> transformerList = transformer.apply(documentList);\n\t\tSystem.out.println(transformerList.size());\n\t}\n\n\t@Test\n\tvoid embed() {\n\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeApi);\n\t\tDocument document = new Document(\"你好阿里云\");\n\t\tfloat[] vectorList = embeddingModel.embed(document);\n\t\tSystem.out.println(vectorList.length);\n\t}\n\n\t@Test\n\tvoid vectorStore() {\n\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n\t\tList<Document> documentList = Arrays.asList(\n\t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_f3cce7cab0b74e3d98a8d684f6fc4b55_10001\", \"abc\", new HashMap<>()));\n\t\t// cloudStore.add(documentList);\n\t\tcloudStore.delete(Arrays.asList(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\"));\n\t\tList<Document> documents = cloudStore.similaritySearch(\"南方电网\");\n\t\tSystem.out.println(documents.size());\n\t}\n\n}", "after_content": "/*\n * Copyright 2023-2024 the original author or authors.\n *\n * Licensed under the Apache License, Version 2.0 (the \"License\");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *      https://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an \"AS IS\" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\n\npackage com.alibaba.cloud.ai.dashscope.chat.client;\n\nimport com.alibaba.cloud.ai.advisor.DocumentRetrievalAdvisor;\nimport com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatOptions;\nimport com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\nimport com.alibaba.cloud.ai.dashscope.embedding.DashScopeEmbeddingModel;\nimport com.alibaba.cloud.ai.dashscope.rag.*;\nimport org.junit.jupiter.api.Assertions;\nimport org.junit.jupiter.api.Test;\nimport org.slf4j.Logger;\nimport org.slf4j.LoggerFactory;\nimport org.springframework.ai.chat.client.ChatClient;\nimport org.springframework.ai.chat.client.advisor.MessageChatMemoryAdvisor;\nimport org.springframework.ai.chat.memory.InMemoryChatMemory;\nimport org.springframework.ai.chat.model.ChatResponse;\nimport org.springframework.ai.document.Document;\nimport org.springframework.ai.document.DocumentRetriever;\nimport org.springframework.beans.factory.annotation.Autowired;\nimport org.springframework.beans.factory.annotation.Value;\nimport org.springframework.boot.test.context.SpringBootTest;\nimport org.springframework.core.io.Resource;\nimport org.springframework.test.context.TestPropertySource;\nimport reactor.core.publisher.Flux;\n\nimport java.io.IOException;\nimport java.nio.charset.StandardCharsets;\nimport java.util.Arrays;\nimport java.util.HashMap;\nimport java.util.List;\nimport java.util.UUID;\nimport java.util.concurrent.CountDownLatch;\n\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_CONVERSATION_ID_KEY;\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_RETRIEVE_SIZE_KEY;\n\n/**\n * Title React agent test cases.<br/>\n * Description React agent test cases.<br/>\n * Created at 2024-08-19 10:15\n *\n * @author yuanci.ytb\n * @version 1.0.0\n * @since jdk8\n */\n\n@TestPropertySource(\"classpath:application.yml\")\n@SpringBootTest(classes = { DashScopeAutoConfiguration.class, DashScopeFunctionTestConfiguration.class,\n\t\tMockOrderService.class })\npublic class DashScopeChatClientIT {\n\n\tprivate static final Logger logger = LoggerFactory.getLogger(DashScopeChatClientIT.class);\n\n\t@Autowired\n\tprivate DashScopeChatModel dashscopeChatModel;\n\n\t@Autowired\n\tprivate DashScopeApi dashscopeChatApi;\n\n\t@Value(\"classpath:/prompts/rag/system-qa.st\")\n\tprivate Resource systemResource;\n\n\t@Value(\"classpath:/prompts/rag/system-qa-ref.st\")\n\tprivate Resource systemResourceRef;\n\n\t@Test\n\tvoid callTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"spring ai alibaba 是什么?\").call().chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt()\n\t\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t\t.options(DashScopeChatOptions.builder()\n\t\t\t\t\t\t.withIncrementalOutput(true)\n\t\t\t\t\t\t.build())\n\t\t\t\t.stream()\n\t\t\t\t.chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithFunctionTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.function(\"getWeather\", \"根据城市查询天气\", new MockWeatherService())\n\t\t\t.user(\"杭州今天的天气如何?\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionBeanTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).defaultFunctions(\"getOrderFunction\").build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.functions(\"getCurrentWeather\")\n\t\t\t.user(\"帮我一下订单, 用户编号为1001, 订单编号为2001\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionAndRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt()\n\t\t\t\t.user(\"上海今天的天气如何?\")\n\t\t\t\t.options(DashScopeChatOptions.builder()\n\t\t\t\t\t\t.withIncrementalOutput(true)\n\t\t\t\t\t\t.build())\n\t\t\t\t.stream()\n\t\t\t\t.chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithReferencedRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t\tList<Document> documents = (List<Document>) response.getMetadata()\n\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\tAssertions.assertNotNull(documents);\n\n\t\tfor (Document document : documents) {\n\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t}\n\t}\n\n\t@Test\n\tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\n\t\t\tChatCompletionFinishReason finishReason = ChatCompletionFinishReason\n\t\t\t\t.valueOf(data.getResult().getMetadata().getFinishReason());\n\t\t\tif (finishReason != ChatCompletionFinishReason.NULL) {\n\t\t\t\tList<Document> documents = (List<Document>) data.getMetadata()\n\t\t\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\t\t\tAssertions.assertNotNull(documents);\n\n\t\t\t\tfor (Document document : documents) {\n\t\t\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t\t\t}\n\t\t\t}\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithMemory() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultAdvisors(new MessageChatMemoryAdvisor(new InMemoryChatMemory()))\n\t\t\t.build();\n\n\t\tString conversantId = UUID.randomUUID().toString();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\n\t\tresponse = chatClient.prompt()\n\t\t\t.user(\"可以给一些使用示例吗?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tcontent = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid reader() {\n\t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeChatApi, null);\n\t\tList<Document> documentList = reader.get();\n\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeChatApi);\n\t\tList<Document> transformerList = transformer.apply(documentList);\n\t\tSystem.out.println(transformerList.size());\n\t}\n\n\t@Test\n\tvoid embed() {\n\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeChatApi);\n\t\tDocument document = new Document(\"你好阿里云\");\n\t\tfloat[] vectorList = embeddingModel.embed(document);\n\t\tSystem.out.println(vectorList.length);\n\t}\n\n\t@Test\n\tvoid vectorStore() {\n\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeChatApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n\t\tList<Document> documentList = Arrays.asList(\n\t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_f3cce7cab0b74e3d98a8d684f6fc4b55_10001\", \"abc\", new HashMap<>()));\n\t\t// cloudStore.add(documentList);\n\t\tcloudStore.delete(Arrays.asList(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\"));\n\t\tList<Document> documents = cloudStore.similaritySearch(\"南方电网\");\n\t\tSystem.out.println(documents.size());\n\t}\n\n}", "patch": "@@ -21,6 +21,7 @@ import com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\n import com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\n import com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\n import com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\n+import com.alibaba.cloud.ai.dashscope.chat.DashScopeChatOptions;\n import com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\n import com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\n import com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\n@@ -75,7 +76,7 @@ public class DashScopeChatClientIT {\n \tprivate DashScopeChatModel dashscopeChatModel;\n \n \t@Autowired\n-\tprivate DashScopeApi dashscopeApi;\n+\tprivate DashScopeApi dashscopeChatApi;\n \n \t@Value(\"classpath:/prompts/rag/system-qa.st\")\n \tprivate Resource systemResource;\n@@ -85,7 +86,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callTest() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -102,14 +103,20 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid streamTest() throws InterruptedException, IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n \t\t\t.defaultAdvisors(\n \t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n \t\t\t.build();\n \n-\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n+\t\tFlux<ChatResponse> response = chatClient.prompt()\n+\t\t\t\t.user(\"如何快速开始百炼?\")\n+\t\t\t\t.options(DashScopeChatOptions.builder()\n+\t\t\t\t\t\t.withIncrementalOutput(true)\n+\t\t\t\t\t\t.build())\n+\t\t\t\t.stream()\n+\t\t\t\t.chatResponse();\n \n \t\tCountDownLatch cdl = new CountDownLatch(1);\n \t\tresponse.subscribe(data -> {\n@@ -159,7 +166,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callWithFunctionAndRagTest() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -178,7 +185,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -187,7 +194,13 @@ public class DashScopeChatClientIT {\n \t\t\t.defaultFunctions(\"weatherFunction\")\n \t\t\t.build();\n \n-\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"上海今天的天气如何?\").stream().chatResponse();\n+\t\tFlux<ChatResponse> response = chatClient.prompt()\n+\t\t\t\t.user(\"上海今天的天气如何?\")\n+\t\t\t\t.options(DashScopeChatOptions.builder()\n+\t\t\t\t\t\t.withIncrementalOutput(true)\n+\t\t\t\t\t\t.build())\n+\t\t\t\t.stream()\n+\t\t\t\t.chatResponse();\n \n \t\tCountDownLatch cdl = new CountDownLatch(1);\n \t\tresponse.subscribe(data -> {\n@@ -206,7 +219,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callWithReferencedRagTest() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -232,7 +245,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -272,7 +285,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callWithMemory() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -309,16 +322,16 @@ public class DashScopeChatClientIT {\n \t@Test\n \tvoid reader() {\n \t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n-\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeApi, null);\n+\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeChatApi, null);\n \t\tList<Document> documentList = reader.get();\n-\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeApi);\n+\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeChatApi);\n \t\tList<Document> transformerList = transformer.apply(documentList);\n \t\tSystem.out.println(transformerList.size());\n \t}\n \n \t@Test\n \tvoid embed() {\n-\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeApi);\n+\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeChatApi);\n \t\tDocument document = new Document(\"你好阿里云\");\n \t\tfloat[] vectorList = embeddingModel.embed(document);\n \t\tSystem.out.println(vectorList.length);\n@@ -326,7 +339,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid vectorStore() {\n-\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n+\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeChatApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n \t\tList<Document> documentList = Arrays.asList(\n \t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n \t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n", "editable_region_start": 20, "editable_region_end": 32, "change_labels": null, "chunk_id": 0, "total_chunks_in_file": 8, "is_multi_chunk_file": true, "model_input_text": "/*\n * Copyright 2023-2024 the original author or authors.\n *\n * Licensed under the Apache License, Version 2.0 (the \"License\");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *      https://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an \"AS IS\" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\n\npackage com.alibaba.cloud.ai.dashscope.chat.client;\n\nimport com.alibaba.cloud.ai.advisor.DocumentRetrievalAdvisor;\n<|editable_region_start|>\nimport com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\nimport com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\nimport com.alibaba.cloud.ai.dashscope.embedding.DashScopeEmbeddingModel;\nimport com.alibaba.cloud.ai.dashscope.rag.*;\nimport org.junit.jupiter.api.Assertions;\nimport org.junit.jupiter.api.Test;\nimport org.slf4j.Logger;\nimport org.slf4j.LoggerFactory;\n<|editable_region_end|>\nimport org.springframework.ai.chat.client.ChatClient;\nimport org.springframework.ai.chat.client.advisor.MessageChatMemoryAdvisor;\nimport org.springframework.ai.chat.memory.InMemoryChatMemory;\nimport org.springframework.ai.chat.model.ChatResponse;\nimport org.springframework.ai.document.Document;\nimport org.springframework.ai.document.DocumentRetriever;\nimport org.springframework.beans.factory.annotation.Autowired;\nimport org.springframework.beans.factory.annotation.Value;\nimport org.springframework.boot.test.context.SpringBootTest;\nimport org.springframework.core.io.Resource;\nimport org.springframework.test.context.TestPropertySource;\nimport reactor.core.publisher.Flux;\n\nimport java.io.IOException;\nimport java.nio.charset.StandardCharsets;\nimport java.util.Arrays;\nimport java.util.HashMap;\nimport java.util.List;\nimport java.util.UUID;\nimport java.util.concurrent.CountDownLatch;\n\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_CONVERSATION_ID_KEY;\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_RETRIEVE_SIZE_KEY;\n\n/**\n * Title React agent test cases.<br/>\n * Description React agent test cases.<br/>\n * Created at 2024-08-19 10:15\n *\n * @author yuanci.ytb\n * @version 1.0.0\n * @since jdk8\n */\n\n@TestPropertySource(\"classpath:application.yml\")\n@SpringBootTest(classes = { DashScopeAutoConfiguration.class, DashScopeFunctionTestConfiguration.class,\n\t\tMockOrderService.class })\npublic class DashScopeChatClientIT {\n\n\tprivate static final Logger logger = LoggerFactory.getLogger(DashScopeChatClientIT.class);\n\n\t@Autowired\n\tprivate DashScopeChatModel dashscopeChatModel;\n\n\t@Autowired\n\tprivate DashScopeApi dashscopeApi;\n\n\t@Value(\"classpath:/prompts/rag/system-qa.st\")\n\tprivate Resource systemResource;\n\n\t@Value(\"classpath:/prompts/rag/system-qa-ref.st\")\n\tprivate Resource systemResourceRef;\n\n\t@Test\n\tvoid callTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"spring ai alibaba 是什么?\").call().chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithFunctionTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.function(\"getWeather\", \"根据城市查询天气\", new MockWeatherService())\n\t\t\t.user(\"杭州今天的天气如何?\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionBeanTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).defaultFunctions(\"getOrderFunction\").build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.functions(\"getCurrentWeather\")\n\t\t\t.user(\"帮我一下订单, 用户编号为1001, 订单编号为2001\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionAndRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"上海今天的天气如何?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithReferencedRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t\tList<Document> documents = (List<Document>) response.getMetadata()\n\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\tAssertions.assertNotNull(documents);\n\n\t\tfor (Document document : documents) {\n\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t}\n\t}\n\n\t@Test\n\tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\n\t\t\tChatCompletionFinishReason finishReason = ChatCompletionFinishReason\n\t\t\t\t.valueOf(data.getResult().getMetadata().getFinishReason());\n\t\t\tif (finishReason != ChatCompletionFinishReason.NULL) {\n\t\t\t\tList<Document> documents = (List<Document>) data.getMetadata()\n\t\t\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\t\t\tAssertions.assertNotNull(documents);\n\n\t\t\t\tfor (Document document : documents) {\n\t\t\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t\t\t}\n\t\t\t}\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithMemory() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultAdvisors(new MessageChatMemoryAdvisor(new InMemoryChatMemory()))\n\t\t\t.build();\n\n\t\tString conversantId = UUID.randomUUID().toString();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\n\t\tresponse = chatClient.prompt()\n\t\t\t.user(\"可以给一些使用示例吗?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tcontent = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid reader() {\n\t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeApi, null);\n\t\tList<Document> documentList = reader.get();\n\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeApi);\n\t\tList<Document> transformerList = transformer.apply(documentList);\n\t\tSystem.out.println(transformerList.size());\n\t}\n\n\t@Test\n\tvoid embed() {\n\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeApi);\n\t\tDocument document = new Document(\"你好阿里云\");\n\t\tfloat[] vectorList = embeddingModel.embed(document);\n\t\tSystem.out.println(vectorList.length);\n\t}\n\n\t@Test\n\tvoid vectorStore() {\n\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n\t\tList<Document> documentList = Arrays.asList(\n\t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_f3cce7cab0b74e3d98a8d684f6fc4b55_10001\", \"abc\", new HashMap<>()));\n\t\t// cloudStore.add(documentList);\n\t\tcloudStore.delete(Arrays.asList(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\"));\n\t\tList<Document> documents = cloudStore.similaritySearch(\"南方电网\");\n\t\tSystem.out.println(documents.size());\n\t}\n\n}", "model_target_text": "import com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatOptions;\nimport com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\nimport com.alibaba.cloud.ai.dashscope.embedding.DashScopeEmbeddingModel;\nimport com.alibaba.cloud.ai.dashscope.rag.*;\nimport org.junit.jupiter.api.Assertions;\nimport org.junit.jupiter.api.Test;\nimport org.slf4j.Logger;", "region_before": "import com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\nimport com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\nimport com.alibaba.cloud.ai.dashscope.embedding.DashScopeEmbeddingModel;\nimport com.alibaba.cloud.ai.dashscope.rag.*;\nimport org.junit.jupiter.api.Assertions;\nimport org.junit.jupiter.api.Test;\nimport org.slf4j.Logger;\nimport org.slf4j.LoggerFactory;", "jaccard_similarity": 0.9310344827586207, "semantic_cosine_similarity": 1.0000001192092896, "llm_judge": {"label": "BAD", "confidence": 5, "reasons": "The target snippet is truncated and incomplete, missing many imports present in the input region. It does not fully cover the editable region and ends abruptly, making it unsuitable as a training target.", "flags": ["TRUNCATED", "MISMATCHED_REGION"]}}
{"commit_sha": "5840b7349a002a847c6c3070276e040b8381e0d0", "commit_short_sha": "5840b734", "commit_message": "fix: incrementalOutput not working (#17)\n\n1. set enable search to false for default chat configuration.\r\n2. Fix incremental output issue and support streaming function call.", "commit_author": "TianBing Ye", "commit_date": "2024-09-23T16:46:24+08:00", "total_files_in_commit": 8, "sequence_order": 7, "file_path": "spring-ai-alibaba-core/src/test/java/com/alibaba/cloud/ai/dashscope/chat/client/DashScopeChatClientIT.java", "file_extension": "java", "change_type": "modified", "lines_added": 27, "lines_deleted": 14, "total_changes": 41, "before_content": "/*\n * Copyright 2023-2024 the original author or authors.\n *\n * Licensed under the Apache License, Version 2.0 (the \"License\");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *      https://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an \"AS IS\" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\n\npackage com.alibaba.cloud.ai.dashscope.chat.client;\n\nimport com.alibaba.cloud.ai.advisor.DocumentRetrievalAdvisor;\nimport com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\nimport com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\nimport com.alibaba.cloud.ai.dashscope.embedding.DashScopeEmbeddingModel;\nimport com.alibaba.cloud.ai.dashscope.rag.*;\nimport org.junit.jupiter.api.Assertions;\nimport org.junit.jupiter.api.Test;\nimport org.slf4j.Logger;\nimport org.slf4j.LoggerFactory;\nimport org.springframework.ai.chat.client.ChatClient;\nimport org.springframework.ai.chat.client.advisor.MessageChatMemoryAdvisor;\nimport org.springframework.ai.chat.memory.InMemoryChatMemory;\nimport org.springframework.ai.chat.model.ChatResponse;\nimport org.springframework.ai.document.Document;\nimport org.springframework.ai.document.DocumentRetriever;\nimport org.springframework.beans.factory.annotation.Autowired;\nimport org.springframework.beans.factory.annotation.Value;\nimport org.springframework.boot.test.context.SpringBootTest;\nimport org.springframework.core.io.Resource;\nimport org.springframework.test.context.TestPropertySource;\nimport reactor.core.publisher.Flux;\n\nimport java.io.IOException;\nimport java.nio.charset.StandardCharsets;\nimport java.util.Arrays;\nimport java.util.HashMap;\nimport java.util.List;\nimport java.util.UUID;\nimport java.util.concurrent.CountDownLatch;\n\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_CONVERSATION_ID_KEY;\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_RETRIEVE_SIZE_KEY;\n\n/**\n * Title React agent test cases.<br/>\n * Description React agent test cases.<br/>\n * Created at 2024-08-19 10:15\n *\n * @author yuanci.ytb\n * @version 1.0.0\n * @since jdk8\n */\n\n@TestPropertySource(\"classpath:application.yml\")\n@SpringBootTest(classes = { DashScopeAutoConfiguration.class, DashScopeFunctionTestConfiguration.class,\n\t\tMockOrderService.class })\npublic class DashScopeChatClientIT {\n\n\tprivate static final Logger logger = LoggerFactory.getLogger(DashScopeChatClientIT.class);\n\n\t@Autowired\n\tprivate DashScopeChatModel dashscopeChatModel;\n\n\t@Autowired\n\tprivate DashScopeApi dashscopeApi;\n\n\t@Value(\"classpath:/prompts/rag/system-qa.st\")\n\tprivate Resource systemResource;\n\n\t@Value(\"classpath:/prompts/rag/system-qa-ref.st\")\n\tprivate Resource systemResourceRef;\n\n\t@Test\n\tvoid callTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"spring ai alibaba 是什么?\").call().chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithFunctionTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.function(\"getWeather\", \"根据城市查询天气\", new MockWeatherService())\n\t\t\t.user(\"杭州今天的天气如何?\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionBeanTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).defaultFunctions(\"getOrderFunction\").build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.functions(\"getCurrentWeather\")\n\t\t\t.user(\"帮我一下订单, 用户编号为1001, 订单编号为2001\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionAndRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"上海今天的天气如何?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithReferencedRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t\tList<Document> documents = (List<Document>) response.getMetadata()\n\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\tAssertions.assertNotNull(documents);\n\n\t\tfor (Document document : documents) {\n\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t}\n\t}\n\n\t@Test\n\tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\n\t\t\tChatCompletionFinishReason finishReason = ChatCompletionFinishReason\n\t\t\t\t.valueOf(data.getResult().getMetadata().getFinishReason());\n\t\t\tif (finishReason != ChatCompletionFinishReason.NULL) {\n\t\t\t\tList<Document> documents = (List<Document>) data.getMetadata()\n\t\t\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\t\t\tAssertions.assertNotNull(documents);\n\n\t\t\t\tfor (Document document : documents) {\n\t\t\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t\t\t}\n\t\t\t}\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithMemory() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultAdvisors(new MessageChatMemoryAdvisor(new InMemoryChatMemory()))\n\t\t\t.build();\n\n\t\tString conversantId = UUID.randomUUID().toString();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\n\t\tresponse = chatClient.prompt()\n\t\t\t.user(\"可以给一些使用示例吗?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tcontent = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid reader() {\n\t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeApi, null);\n\t\tList<Document> documentList = reader.get();\n\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeApi);\n\t\tList<Document> transformerList = transformer.apply(documentList);\n\t\tSystem.out.println(transformerList.size());\n\t}\n\n\t@Test\n\tvoid embed() {\n\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeApi);\n\t\tDocument document = new Document(\"你好阿里云\");\n\t\tfloat[] vectorList = embeddingModel.embed(document);\n\t\tSystem.out.println(vectorList.length);\n\t}\n\n\t@Test\n\tvoid vectorStore() {\n\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n\t\tList<Document> documentList = Arrays.asList(\n\t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_f3cce7cab0b74e3d98a8d684f6fc4b55_10001\", \"abc\", new HashMap<>()));\n\t\t// cloudStore.add(documentList);\n\t\tcloudStore.delete(Arrays.asList(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\"));\n\t\tList<Document> documents = cloudStore.similaritySearch(\"南方电网\");\n\t\tSystem.out.println(documents.size());\n\t}\n\n}", "after_content": "/*\n * Copyright 2023-2024 the original author or authors.\n *\n * Licensed under the Apache License, Version 2.0 (the \"License\");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *      https://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an \"AS IS\" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\n\npackage com.alibaba.cloud.ai.dashscope.chat.client;\n\nimport com.alibaba.cloud.ai.advisor.DocumentRetrievalAdvisor;\nimport com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatOptions;\nimport com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\nimport com.alibaba.cloud.ai.dashscope.embedding.DashScopeEmbeddingModel;\nimport com.alibaba.cloud.ai.dashscope.rag.*;\nimport org.junit.jupiter.api.Assertions;\nimport org.junit.jupiter.api.Test;\nimport org.slf4j.Logger;\nimport org.slf4j.LoggerFactory;\nimport org.springframework.ai.chat.client.ChatClient;\nimport org.springframework.ai.chat.client.advisor.MessageChatMemoryAdvisor;\nimport org.springframework.ai.chat.memory.InMemoryChatMemory;\nimport org.springframework.ai.chat.model.ChatResponse;\nimport org.springframework.ai.document.Document;\nimport org.springframework.ai.document.DocumentRetriever;\nimport org.springframework.beans.factory.annotation.Autowired;\nimport org.springframework.beans.factory.annotation.Value;\nimport org.springframework.boot.test.context.SpringBootTest;\nimport org.springframework.core.io.Resource;\nimport org.springframework.test.context.TestPropertySource;\nimport reactor.core.publisher.Flux;\n\nimport java.io.IOException;\nimport java.nio.charset.StandardCharsets;\nimport java.util.Arrays;\nimport java.util.HashMap;\nimport java.util.List;\nimport java.util.UUID;\nimport java.util.concurrent.CountDownLatch;\n\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_CONVERSATION_ID_KEY;\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_RETRIEVE_SIZE_KEY;\n\n/**\n * Title React agent test cases.<br/>\n * Description React agent test cases.<br/>\n * Created at 2024-08-19 10:15\n *\n * @author yuanci.ytb\n * @version 1.0.0\n * @since jdk8\n */\n\n@TestPropertySource(\"classpath:application.yml\")\n@SpringBootTest(classes = { DashScopeAutoConfiguration.class, DashScopeFunctionTestConfiguration.class,\n\t\tMockOrderService.class })\npublic class DashScopeChatClientIT {\n\n\tprivate static final Logger logger = LoggerFactory.getLogger(DashScopeChatClientIT.class);\n\n\t@Autowired\n\tprivate DashScopeChatModel dashscopeChatModel;\n\n\t@Autowired\n\tprivate DashScopeApi dashscopeChatApi;\n\n\t@Value(\"classpath:/prompts/rag/system-qa.st\")\n\tprivate Resource systemResource;\n\n\t@Value(\"classpath:/prompts/rag/system-qa-ref.st\")\n\tprivate Resource systemResourceRef;\n\n\t@Test\n\tvoid callTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"spring ai alibaba 是什么?\").call().chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt()\n\t\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t\t.options(DashScopeChatOptions.builder()\n\t\t\t\t\t\t.withIncrementalOutput(true)\n\t\t\t\t\t\t.build())\n\t\t\t\t.stream()\n\t\t\t\t.chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithFunctionTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.function(\"getWeather\", \"根据城市查询天气\", new MockWeatherService())\n\t\t\t.user(\"杭州今天的天气如何?\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionBeanTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).defaultFunctions(\"getOrderFunction\").build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.functions(\"getCurrentWeather\")\n\t\t\t.user(\"帮我一下订单, 用户编号为1001, 订单编号为2001\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionAndRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt()\n\t\t\t\t.user(\"上海今天的天气如何?\")\n\t\t\t\t.options(DashScopeChatOptions.builder()\n\t\t\t\t\t\t.withIncrementalOutput(true)\n\t\t\t\t\t\t.build())\n\t\t\t\t.stream()\n\t\t\t\t.chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithReferencedRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t\tList<Document> documents = (List<Document>) response.getMetadata()\n\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\tAssertions.assertNotNull(documents);\n\n\t\tfor (Document document : documents) {\n\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t}\n\t}\n\n\t@Test\n\tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\n\t\t\tChatCompletionFinishReason finishReason = ChatCompletionFinishReason\n\t\t\t\t.valueOf(data.getResult().getMetadata().getFinishReason());\n\t\t\tif (finishReason != ChatCompletionFinishReason.NULL) {\n\t\t\t\tList<Document> documents = (List<Document>) data.getMetadata()\n\t\t\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\t\t\tAssertions.assertNotNull(documents);\n\n\t\t\t\tfor (Document document : documents) {\n\t\t\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t\t\t}\n\t\t\t}\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithMemory() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultAdvisors(new MessageChatMemoryAdvisor(new InMemoryChatMemory()))\n\t\t\t.build();\n\n\t\tString conversantId = UUID.randomUUID().toString();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\n\t\tresponse = chatClient.prompt()\n\t\t\t.user(\"可以给一些使用示例吗?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tcontent = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid reader() {\n\t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeChatApi, null);\n\t\tList<Document> documentList = reader.get();\n\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeChatApi);\n\t\tList<Document> transformerList = transformer.apply(documentList);\n\t\tSystem.out.println(transformerList.size());\n\t}\n\n\t@Test\n\tvoid embed() {\n\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeChatApi);\n\t\tDocument document = new Document(\"你好阿里云\");\n\t\tfloat[] vectorList = embeddingModel.embed(document);\n\t\tSystem.out.println(vectorList.length);\n\t}\n\n\t@Test\n\tvoid vectorStore() {\n\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeChatApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n\t\tList<Document> documentList = Arrays.asList(\n\t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_f3cce7cab0b74e3d98a8d684f6fc4b55_10001\", \"abc\", new HashMap<>()));\n\t\t// cloudStore.add(documentList);\n\t\tcloudStore.delete(Arrays.asList(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\"));\n\t\tList<Document> documents = cloudStore.similaritySearch(\"南方电网\");\n\t\tSystem.out.println(documents.size());\n\t}\n\n}", "patch": "@@ -21,6 +21,7 @@ import com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\n import com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\n import com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\n import com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\n+import com.alibaba.cloud.ai.dashscope.chat.DashScopeChatOptions;\n import com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\n import com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\n import com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\n@@ -75,7 +76,7 @@ public class DashScopeChatClientIT {\n \tprivate DashScopeChatModel dashscopeChatModel;\n \n \t@Autowired\n-\tprivate DashScopeApi dashscopeApi;\n+\tprivate DashScopeApi dashscopeChatApi;\n \n \t@Value(\"classpath:/prompts/rag/system-qa.st\")\n \tprivate Resource systemResource;\n@@ -85,7 +86,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callTest() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -102,14 +103,20 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid streamTest() throws InterruptedException, IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n \t\t\t.defaultAdvisors(\n \t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n \t\t\t.build();\n \n-\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n+\t\tFlux<ChatResponse> response = chatClient.prompt()\n+\t\t\t\t.user(\"如何快速开始百炼?\")\n+\t\t\t\t.options(DashScopeChatOptions.builder()\n+\t\t\t\t\t\t.withIncrementalOutput(true)\n+\t\t\t\t\t\t.build())\n+\t\t\t\t.stream()\n+\t\t\t\t.chatResponse();\n \n \t\tCountDownLatch cdl = new CountDownLatch(1);\n \t\tresponse.subscribe(data -> {\n@@ -159,7 +166,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callWithFunctionAndRagTest() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -178,7 +185,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -187,7 +194,13 @@ public class DashScopeChatClientIT {\n \t\t\t.defaultFunctions(\"weatherFunction\")\n \t\t\t.build();\n \n-\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"上海今天的天气如何?\").stream().chatResponse();\n+\t\tFlux<ChatResponse> response = chatClient.prompt()\n+\t\t\t\t.user(\"上海今天的天气如何?\")\n+\t\t\t\t.options(DashScopeChatOptions.builder()\n+\t\t\t\t\t\t.withIncrementalOutput(true)\n+\t\t\t\t\t\t.build())\n+\t\t\t\t.stream()\n+\t\t\t\t.chatResponse();\n \n \t\tCountDownLatch cdl = new CountDownLatch(1);\n \t\tresponse.subscribe(data -> {\n@@ -206,7 +219,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callWithReferencedRagTest() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -232,7 +245,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -272,7 +285,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callWithMemory() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -309,16 +322,16 @@ public class DashScopeChatClientIT {\n \t@Test\n \tvoid reader() {\n \t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n-\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeApi, null);\n+\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeChatApi, null);\n \t\tList<Document> documentList = reader.get();\n-\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeApi);\n+\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeChatApi);\n \t\tList<Document> transformerList = transformer.apply(documentList);\n \t\tSystem.out.println(transformerList.size());\n \t}\n \n \t@Test\n \tvoid embed() {\n-\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeApi);\n+\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeChatApi);\n \t\tDocument document = new Document(\"你好阿里云\");\n \t\tfloat[] vectorList = embeddingModel.embed(document);\n \t\tSystem.out.println(vectorList.length);\n@@ -326,7 +339,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid vectorStore() {\n-\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n+\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeChatApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n \t\tList<Document> documentList = Arrays.asList(\n \t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n \t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n", "editable_region_start": 74, "editable_region_end": 121, "change_labels": null, "chunk_id": 1, "total_chunks_in_file": 8, "is_multi_chunk_file": true, "model_input_text": "/*\n * Copyright 2023-2024 the original author or authors.\n *\n * Licensed under the Apache License, Version 2.0 (the \"License\");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *      https://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an \"AS IS\" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\n\npackage com.alibaba.cloud.ai.dashscope.chat.client;\n\nimport com.alibaba.cloud.ai.advisor.DocumentRetrievalAdvisor;\nimport com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\nimport com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\nimport com.alibaba.cloud.ai.dashscope.embedding.DashScopeEmbeddingModel;\nimport com.alibaba.cloud.ai.dashscope.rag.*;\nimport org.junit.jupiter.api.Assertions;\nimport org.junit.jupiter.api.Test;\nimport org.slf4j.Logger;\nimport org.slf4j.LoggerFactory;\nimport org.springframework.ai.chat.client.ChatClient;\nimport org.springframework.ai.chat.client.advisor.MessageChatMemoryAdvisor;\nimport org.springframework.ai.chat.memory.InMemoryChatMemory;\nimport org.springframework.ai.chat.model.ChatResponse;\nimport org.springframework.ai.document.Document;\nimport org.springframework.ai.document.DocumentRetriever;\nimport org.springframework.beans.factory.annotation.Autowired;\nimport org.springframework.beans.factory.annotation.Value;\nimport org.springframework.boot.test.context.SpringBootTest;\nimport org.springframework.core.io.Resource;\nimport org.springframework.test.context.TestPropertySource;\nimport reactor.core.publisher.Flux;\n\nimport java.io.IOException;\nimport java.nio.charset.StandardCharsets;\nimport java.util.Arrays;\nimport java.util.HashMap;\nimport java.util.List;\nimport java.util.UUID;\nimport java.util.concurrent.CountDownLatch;\n\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_CONVERSATION_ID_KEY;\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_RETRIEVE_SIZE_KEY;\n\n/**\n * Title React agent test cases.<br/>\n * Description React agent test cases.<br/>\n * Created at 2024-08-19 10:15\n *\n * @author yuanci.ytb\n * @version 1.0.0\n * @since jdk8\n */\n\n@TestPropertySource(\"classpath:application.yml\")\n@SpringBootTest(classes = { DashScopeAutoConfiguration.class, DashScopeFunctionTestConfiguration.class,\n\t\tMockOrderService.class })\npublic class DashScopeChatClientIT {\n\n\tprivate static final Logger logger = LoggerFactory.getLogger(DashScopeChatClientIT.class);\n\n<|editable_region_start|>\n\t@Autowired\n\tprivate DashScopeChatModel dashscopeChatModel;\n\n\t@Autowired\n\tprivate DashScopeApi dashscopeApi;\n\n\t@Value(\"classpath:/prompts/rag/system-qa.st\")\n\tprivate Resource systemResource;\n\n\t@Value(\"classpath:/prompts/rag/system-qa-ref.st\")\n\tprivate Resource systemResourceRef;\n\n\t@Test\n\tvoid callTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"spring ai alibaba 是什么?\").call().chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tlogger.info(\"done\");\n<|editable_region_end|>\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithFunctionTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.function(\"getWeather\", \"根据城市查询天气\", new MockWeatherService())\n\t\t\t.user(\"杭州今天的天气如何?\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionBeanTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).defaultFunctions(\"getOrderFunction\").build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.functions(\"getCurrentWeather\")\n\t\t\t.user(\"帮我一下订单, 用户编号为1001, 订单编号为2001\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionAndRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"上海今天的天气如何?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithReferencedRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t\tList<Document> documents = (List<Document>) response.getMetadata()\n\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\tAssertions.assertNotNull(documents);\n\n\t\tfor (Document document : documents) {\n\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t}\n\t}\n\n\t@Test\n\tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\n\t\t\tChatCompletionFinishReason finishReason = ChatCompletionFinishReason\n\t\t\t\t.valueOf(data.getResult().getMetadata().getFinishReason());\n\t\t\tif (finishReason != ChatCompletionFinishReason.NULL) {\n\t\t\t\tList<Document> documents = (List<Document>) data.getMetadata()\n\t\t\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\t\t\tAssertions.assertNotNull(documents);\n\n\t\t\t\tfor (Document document : documents) {\n\t\t\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t\t\t}\n\t\t\t}\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithMemory() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultAdvisors(new MessageChatMemoryAdvisor(new InMemoryChatMemory()))\n\t\t\t.build();\n\n\t\tString conversantId = UUID.randomUUID().toString();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\n\t\tresponse = chatClient.prompt()\n\t\t\t.user(\"可以给一些使用示例吗?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tcontent = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid reader() {\n\t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeApi, null);\n\t\tList<Document> documentList = reader.get();\n\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeApi);\n\t\tList<Document> transformerList = transformer.apply(documentList);\n\t\tSystem.out.println(transformerList.size());\n\t}\n\n\t@Test\n\tvoid embed() {\n\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeApi);\n\t\tDocument document = new Document(\"你好阿里云\");\n\t\tfloat[] vectorList = embeddingModel.embed(document);\n\t\tSystem.out.println(vectorList.length);\n\t}\n\n\t@Test\n\tvoid vectorStore() {\n\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n\t\tList<Document> documentList = Arrays.asList(\n\t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_f3cce7cab0b74e3d98a8d684f6fc4b55_10001\", \"abc\", new HashMap<>()));\n\t\t// cloudStore.add(documentList);\n\t\tcloudStore.delete(Arrays.asList(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\"));\n\t\tList<Document> documents = cloudStore.similaritySearch(\"南方电网\");\n\t\tSystem.out.println(documents.size());\n\t}\n\n}", "model_target_text": "\n\t@Autowired\n\tprivate DashScopeChatModel dashscopeChatModel;\n\n\t@Autowired\n\tprivate DashScopeApi dashscopeChatApi;\n\n\t@Value(\"classpath:/prompts/rag/system-qa.st\")\n\tprivate Resource systemResource;\n\n\t@Value(\"classpath:/prompts/rag/system-qa-ref.st\")\n\tprivate Resource systemResourceRef;\n\n\t@Test\n\tvoid callTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"spring ai alibaba 是什么?\").call().chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt()\n\t\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t\t.options(DashScopeChatOptions.builder()\n\t\t\t\t\t\t.withIncrementalOutput(true)\n\t\t\t\t\t\t.build())\n\t\t\t\t.stream()\n\t\t\t\t.chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);", "region_before": "@Autowired\n\tprivate DashScopeChatModel dashscopeChatModel;\n\n\t@Autowired\n\tprivate DashScopeApi dashscopeApi;\n\n\t@Value(\"classpath:/prompts/rag/system-qa.st\")\n\tprivate Resource systemResource;\n\n\t@Value(\"classpath:/prompts/rag/system-qa-ref.st\")\n\tprivate Resource systemResourceRef;\n\n\t@Test\n\tvoid callTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"spring ai alibaba 是什么?\").call().chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tlogger.info(\"done\");", "jaccard_similarity": 0.7901234567901234, "semantic_cosine_similarity": 1.0000001192092896, "llm_judge": {"label": "BAD", "confidence": 5, "reasons": "The target snippet is truncated and incomplete, ending abruptly inside the streamTest method. It also uses dashscopeChatApi instead of dashscopeApi, which is inconsistent with the input. The target does not fully cover the editable region and is missing large parts of the code, making it unsuitable as training data.", "flags": ["TRUNCATED", "MISMATCHED_REGION"]}}
{"commit_sha": "5840b7349a002a847c6c3070276e040b8381e0d0", "commit_short_sha": "5840b734", "commit_message": "fix: incrementalOutput not working (#17)\n\n1. set enable search to false for default chat configuration.\r\n2. Fix incremental output issue and support streaming function call.", "commit_author": "TianBing Ye", "commit_date": "2024-09-23T16:46:24+08:00", "total_files_in_commit": 8, "sequence_order": 7, "file_path": "spring-ai-alibaba-core/src/test/java/com/alibaba/cloud/ai/dashscope/chat/client/DashScopeChatClientIT.java", "file_extension": "java", "change_type": "modified", "lines_added": 27, "lines_deleted": 14, "total_changes": 41, "before_content": "/*\n * Copyright 2023-2024 the original author or authors.\n *\n * Licensed under the Apache License, Version 2.0 (the \"License\");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *      https://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an \"AS IS\" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\n\npackage com.alibaba.cloud.ai.dashscope.chat.client;\n\nimport com.alibaba.cloud.ai.advisor.DocumentRetrievalAdvisor;\nimport com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\nimport com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\nimport com.alibaba.cloud.ai.dashscope.embedding.DashScopeEmbeddingModel;\nimport com.alibaba.cloud.ai.dashscope.rag.*;\nimport org.junit.jupiter.api.Assertions;\nimport org.junit.jupiter.api.Test;\nimport org.slf4j.Logger;\nimport org.slf4j.LoggerFactory;\nimport org.springframework.ai.chat.client.ChatClient;\nimport org.springframework.ai.chat.client.advisor.MessageChatMemoryAdvisor;\nimport org.springframework.ai.chat.memory.InMemoryChatMemory;\nimport org.springframework.ai.chat.model.ChatResponse;\nimport org.springframework.ai.document.Document;\nimport org.springframework.ai.document.DocumentRetriever;\nimport org.springframework.beans.factory.annotation.Autowired;\nimport org.springframework.beans.factory.annotation.Value;\nimport org.springframework.boot.test.context.SpringBootTest;\nimport org.springframework.core.io.Resource;\nimport org.springframework.test.context.TestPropertySource;\nimport reactor.core.publisher.Flux;\n\nimport java.io.IOException;\nimport java.nio.charset.StandardCharsets;\nimport java.util.Arrays;\nimport java.util.HashMap;\nimport java.util.List;\nimport java.util.UUID;\nimport java.util.concurrent.CountDownLatch;\n\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_CONVERSATION_ID_KEY;\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_RETRIEVE_SIZE_KEY;\n\n/**\n * Title React agent test cases.<br/>\n * Description React agent test cases.<br/>\n * Created at 2024-08-19 10:15\n *\n * @author yuanci.ytb\n * @version 1.0.0\n * @since jdk8\n */\n\n@TestPropertySource(\"classpath:application.yml\")\n@SpringBootTest(classes = { DashScopeAutoConfiguration.class, DashScopeFunctionTestConfiguration.class,\n\t\tMockOrderService.class })\npublic class DashScopeChatClientIT {\n\n\tprivate static final Logger logger = LoggerFactory.getLogger(DashScopeChatClientIT.class);\n\n\t@Autowired\n\tprivate DashScopeChatModel dashscopeChatModel;\n\n\t@Autowired\n\tprivate DashScopeApi dashscopeApi;\n\n\t@Value(\"classpath:/prompts/rag/system-qa.st\")\n\tprivate Resource systemResource;\n\n\t@Value(\"classpath:/prompts/rag/system-qa-ref.st\")\n\tprivate Resource systemResourceRef;\n\n\t@Test\n\tvoid callTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"spring ai alibaba 是什么?\").call().chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithFunctionTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.function(\"getWeather\", \"根据城市查询天气\", new MockWeatherService())\n\t\t\t.user(\"杭州今天的天气如何?\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionBeanTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).defaultFunctions(\"getOrderFunction\").build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.functions(\"getCurrentWeather\")\n\t\t\t.user(\"帮我一下订单, 用户编号为1001, 订单编号为2001\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionAndRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"上海今天的天气如何?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithReferencedRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t\tList<Document> documents = (List<Document>) response.getMetadata()\n\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\tAssertions.assertNotNull(documents);\n\n\t\tfor (Document document : documents) {\n\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t}\n\t}\n\n\t@Test\n\tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\n\t\t\tChatCompletionFinishReason finishReason = ChatCompletionFinishReason\n\t\t\t\t.valueOf(data.getResult().getMetadata().getFinishReason());\n\t\t\tif (finishReason != ChatCompletionFinishReason.NULL) {\n\t\t\t\tList<Document> documents = (List<Document>) data.getMetadata()\n\t\t\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\t\t\tAssertions.assertNotNull(documents);\n\n\t\t\t\tfor (Document document : documents) {\n\t\t\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t\t\t}\n\t\t\t}\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithMemory() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultAdvisors(new MessageChatMemoryAdvisor(new InMemoryChatMemory()))\n\t\t\t.build();\n\n\t\tString conversantId = UUID.randomUUID().toString();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\n\t\tresponse = chatClient.prompt()\n\t\t\t.user(\"可以给一些使用示例吗?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tcontent = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid reader() {\n\t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeApi, null);\n\t\tList<Document> documentList = reader.get();\n\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeApi);\n\t\tList<Document> transformerList = transformer.apply(documentList);\n\t\tSystem.out.println(transformerList.size());\n\t}\n\n\t@Test\n\tvoid embed() {\n\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeApi);\n\t\tDocument document = new Document(\"你好阿里云\");\n\t\tfloat[] vectorList = embeddingModel.embed(document);\n\t\tSystem.out.println(vectorList.length);\n\t}\n\n\t@Test\n\tvoid vectorStore() {\n\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n\t\tList<Document> documentList = Arrays.asList(\n\t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_f3cce7cab0b74e3d98a8d684f6fc4b55_10001\", \"abc\", new HashMap<>()));\n\t\t// cloudStore.add(documentList);\n\t\tcloudStore.delete(Arrays.asList(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\"));\n\t\tList<Document> documents = cloudStore.similaritySearch(\"南方电网\");\n\t\tSystem.out.println(documents.size());\n\t}\n\n}", "after_content": "/*\n * Copyright 2023-2024 the original author or authors.\n *\n * Licensed under the Apache License, Version 2.0 (the \"License\");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *      https://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an \"AS IS\" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\n\npackage com.alibaba.cloud.ai.dashscope.chat.client;\n\nimport com.alibaba.cloud.ai.advisor.DocumentRetrievalAdvisor;\nimport com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatOptions;\nimport com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\nimport com.alibaba.cloud.ai.dashscope.embedding.DashScopeEmbeddingModel;\nimport com.alibaba.cloud.ai.dashscope.rag.*;\nimport org.junit.jupiter.api.Assertions;\nimport org.junit.jupiter.api.Test;\nimport org.slf4j.Logger;\nimport org.slf4j.LoggerFactory;\nimport org.springframework.ai.chat.client.ChatClient;\nimport org.springframework.ai.chat.client.advisor.MessageChatMemoryAdvisor;\nimport org.springframework.ai.chat.memory.InMemoryChatMemory;\nimport org.springframework.ai.chat.model.ChatResponse;\nimport org.springframework.ai.document.Document;\nimport org.springframework.ai.document.DocumentRetriever;\nimport org.springframework.beans.factory.annotation.Autowired;\nimport org.springframework.beans.factory.annotation.Value;\nimport org.springframework.boot.test.context.SpringBootTest;\nimport org.springframework.core.io.Resource;\nimport org.springframework.test.context.TestPropertySource;\nimport reactor.core.publisher.Flux;\n\nimport java.io.IOException;\nimport java.nio.charset.StandardCharsets;\nimport java.util.Arrays;\nimport java.util.HashMap;\nimport java.util.List;\nimport java.util.UUID;\nimport java.util.concurrent.CountDownLatch;\n\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_CONVERSATION_ID_KEY;\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_RETRIEVE_SIZE_KEY;\n\n/**\n * Title React agent test cases.<br/>\n * Description React agent test cases.<br/>\n * Created at 2024-08-19 10:15\n *\n * @author yuanci.ytb\n * @version 1.0.0\n * @since jdk8\n */\n\n@TestPropertySource(\"classpath:application.yml\")\n@SpringBootTest(classes = { DashScopeAutoConfiguration.class, DashScopeFunctionTestConfiguration.class,\n\t\tMockOrderService.class })\npublic class DashScopeChatClientIT {\n\n\tprivate static final Logger logger = LoggerFactory.getLogger(DashScopeChatClientIT.class);\n\n\t@Autowired\n\tprivate DashScopeChatModel dashscopeChatModel;\n\n\t@Autowired\n\tprivate DashScopeApi dashscopeChatApi;\n\n\t@Value(\"classpath:/prompts/rag/system-qa.st\")\n\tprivate Resource systemResource;\n\n\t@Value(\"classpath:/prompts/rag/system-qa-ref.st\")\n\tprivate Resource systemResourceRef;\n\n\t@Test\n\tvoid callTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"spring ai alibaba 是什么?\").call().chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt()\n\t\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t\t.options(DashScopeChatOptions.builder()\n\t\t\t\t\t\t.withIncrementalOutput(true)\n\t\t\t\t\t\t.build())\n\t\t\t\t.stream()\n\t\t\t\t.chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithFunctionTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.function(\"getWeather\", \"根据城市查询天气\", new MockWeatherService())\n\t\t\t.user(\"杭州今天的天气如何?\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionBeanTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).defaultFunctions(\"getOrderFunction\").build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.functions(\"getCurrentWeather\")\n\t\t\t.user(\"帮我一下订单, 用户编号为1001, 订单编号为2001\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionAndRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt()\n\t\t\t\t.user(\"上海今天的天气如何?\")\n\t\t\t\t.options(DashScopeChatOptions.builder()\n\t\t\t\t\t\t.withIncrementalOutput(true)\n\t\t\t\t\t\t.build())\n\t\t\t\t.stream()\n\t\t\t\t.chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithReferencedRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t\tList<Document> documents = (List<Document>) response.getMetadata()\n\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\tAssertions.assertNotNull(documents);\n\n\t\tfor (Document document : documents) {\n\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t}\n\t}\n\n\t@Test\n\tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\n\t\t\tChatCompletionFinishReason finishReason = ChatCompletionFinishReason\n\t\t\t\t.valueOf(data.getResult().getMetadata().getFinishReason());\n\t\t\tif (finishReason != ChatCompletionFinishReason.NULL) {\n\t\t\t\tList<Document> documents = (List<Document>) data.getMetadata()\n\t\t\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\t\t\tAssertions.assertNotNull(documents);\n\n\t\t\t\tfor (Document document : documents) {\n\t\t\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t\t\t}\n\t\t\t}\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithMemory() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultAdvisors(new MessageChatMemoryAdvisor(new InMemoryChatMemory()))\n\t\t\t.build();\n\n\t\tString conversantId = UUID.randomUUID().toString();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\n\t\tresponse = chatClient.prompt()\n\t\t\t.user(\"可以给一些使用示例吗?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tcontent = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid reader() {\n\t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeChatApi, null);\n\t\tList<Document> documentList = reader.get();\n\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeChatApi);\n\t\tList<Document> transformerList = transformer.apply(documentList);\n\t\tSystem.out.println(transformerList.size());\n\t}\n\n\t@Test\n\tvoid embed() {\n\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeChatApi);\n\t\tDocument document = new Document(\"你好阿里云\");\n\t\tfloat[] vectorList = embeddingModel.embed(document);\n\t\tSystem.out.println(vectorList.length);\n\t}\n\n\t@Test\n\tvoid vectorStore() {\n\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeChatApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n\t\tList<Document> documentList = Arrays.asList(\n\t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_f3cce7cab0b74e3d98a8d684f6fc4b55_10001\", \"abc\", new HashMap<>()));\n\t\t// cloudStore.add(documentList);\n\t\tcloudStore.delete(Arrays.asList(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\"));\n\t\tList<Document> documents = cloudStore.similaritySearch(\"南方电网\");\n\t\tSystem.out.println(documents.size());\n\t}\n\n}", "patch": "@@ -21,6 +21,7 @@ import com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\n import com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\n import com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\n import com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\n+import com.alibaba.cloud.ai.dashscope.chat.DashScopeChatOptions;\n import com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\n import com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\n import com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\n@@ -75,7 +76,7 @@ public class DashScopeChatClientIT {\n \tprivate DashScopeChatModel dashscopeChatModel;\n \n \t@Autowired\n-\tprivate DashScopeApi dashscopeApi;\n+\tprivate DashScopeApi dashscopeChatApi;\n \n \t@Value(\"classpath:/prompts/rag/system-qa.st\")\n \tprivate Resource systemResource;\n@@ -85,7 +86,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callTest() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -102,14 +103,20 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid streamTest() throws InterruptedException, IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n \t\t\t.defaultAdvisors(\n \t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n \t\t\t.build();\n \n-\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n+\t\tFlux<ChatResponse> response = chatClient.prompt()\n+\t\t\t\t.user(\"如何快速开始百炼?\")\n+\t\t\t\t.options(DashScopeChatOptions.builder()\n+\t\t\t\t\t\t.withIncrementalOutput(true)\n+\t\t\t\t\t\t.build())\n+\t\t\t\t.stream()\n+\t\t\t\t.chatResponse();\n \n \t\tCountDownLatch cdl = new CountDownLatch(1);\n \t\tresponse.subscribe(data -> {\n@@ -159,7 +166,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callWithFunctionAndRagTest() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -178,7 +185,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -187,7 +194,13 @@ public class DashScopeChatClientIT {\n \t\t\t.defaultFunctions(\"weatherFunction\")\n \t\t\t.build();\n \n-\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"上海今天的天气如何?\").stream().chatResponse();\n+\t\tFlux<ChatResponse> response = chatClient.prompt()\n+\t\t\t\t.user(\"上海今天的天气如何?\")\n+\t\t\t\t.options(DashScopeChatOptions.builder()\n+\t\t\t\t\t\t.withIncrementalOutput(true)\n+\t\t\t\t\t\t.build())\n+\t\t\t\t.stream()\n+\t\t\t\t.chatResponse();\n \n \t\tCountDownLatch cdl = new CountDownLatch(1);\n \t\tresponse.subscribe(data -> {\n@@ -206,7 +219,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callWithReferencedRagTest() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -232,7 +245,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -272,7 +285,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callWithMemory() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -309,16 +322,16 @@ public class DashScopeChatClientIT {\n \t@Test\n \tvoid reader() {\n \t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n-\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeApi, null);\n+\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeChatApi, null);\n \t\tList<Document> documentList = reader.get();\n-\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeApi);\n+\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeChatApi);\n \t\tList<Document> transformerList = transformer.apply(documentList);\n \t\tSystem.out.println(transformerList.size());\n \t}\n \n \t@Test\n \tvoid embed() {\n-\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeApi);\n+\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeChatApi);\n \t\tDocument document = new Document(\"你好阿里云\");\n \t\tfloat[] vectorList = embeddingModel.embed(document);\n \t\tSystem.out.println(vectorList.length);\n@@ -326,7 +339,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid vectorStore() {\n-\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n+\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeChatApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n \t\tList<Document> documentList = Arrays.asList(\n \t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n \t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n", "editable_region_start": 158, "editable_region_end": 171, "change_labels": null, "chunk_id": 2, "total_chunks_in_file": 8, "is_multi_chunk_file": true, "model_input_text": "/*\n * Copyright 2023-2024 the original author or authors.\n *\n * Licensed under the Apache License, Version 2.0 (the \"License\");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *      https://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an \"AS IS\" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\n\npackage com.alibaba.cloud.ai.dashscope.chat.client;\n\nimport com.alibaba.cloud.ai.advisor.DocumentRetrievalAdvisor;\nimport com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\nimport com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\nimport com.alibaba.cloud.ai.dashscope.embedding.DashScopeEmbeddingModel;\nimport com.alibaba.cloud.ai.dashscope.rag.*;\nimport org.junit.jupiter.api.Assertions;\nimport org.junit.jupiter.api.Test;\nimport org.slf4j.Logger;\nimport org.slf4j.LoggerFactory;\nimport org.springframework.ai.chat.client.ChatClient;\nimport org.springframework.ai.chat.client.advisor.MessageChatMemoryAdvisor;\nimport org.springframework.ai.chat.memory.InMemoryChatMemory;\nimport org.springframework.ai.chat.model.ChatResponse;\nimport org.springframework.ai.document.Document;\nimport org.springframework.ai.document.DocumentRetriever;\nimport org.springframework.beans.factory.annotation.Autowired;\nimport org.springframework.beans.factory.annotation.Value;\nimport org.springframework.boot.test.context.SpringBootTest;\nimport org.springframework.core.io.Resource;\nimport org.springframework.test.context.TestPropertySource;\nimport reactor.core.publisher.Flux;\n\nimport java.io.IOException;\nimport java.nio.charset.StandardCharsets;\nimport java.util.Arrays;\nimport java.util.HashMap;\nimport java.util.List;\nimport java.util.UUID;\nimport java.util.concurrent.CountDownLatch;\n\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_CONVERSATION_ID_KEY;\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_RETRIEVE_SIZE_KEY;\n\n/**\n * Title React agent test cases.<br/>\n * Description React agent test cases.<br/>\n * Created at 2024-08-19 10:15\n *\n * @author yuanci.ytb\n * @version 1.0.0\n * @since jdk8\n */\n\n@TestPropertySource(\"classpath:application.yml\")\n@SpringBootTest(classes = { DashScopeAutoConfiguration.class, DashScopeFunctionTestConfiguration.class,\n\t\tMockOrderService.class })\npublic class DashScopeChatClientIT {\n\n\tprivate static final Logger logger = LoggerFactory.getLogger(DashScopeChatClientIT.class);\n\n\t@Autowired\n\tprivate DashScopeChatModel dashscopeChatModel;\n\n\t@Autowired\n\tprivate DashScopeApi dashscopeApi;\n\n\t@Value(\"classpath:/prompts/rag/system-qa.st\")\n\tprivate Resource systemResource;\n\n\t@Value(\"classpath:/prompts/rag/system-qa-ref.st\")\n\tprivate Resource systemResourceRef;\n\n\t@Test\n\tvoid callTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"spring ai alibaba 是什么?\").call().chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithFunctionTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.function(\"getWeather\", \"根据城市查询天气\", new MockWeatherService())\n\t\t\t.user(\"杭州今天的天气如何?\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionBeanTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).defaultFunctions(\"getOrderFunction\").build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.functions(\"getCurrentWeather\")\n\t\t\t.user(\"帮我一下订单, 用户编号为1001, 订单编号为2001\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n<|editable_region_start|>\n\t}\n\n\t@Test\n\tvoid callWithFunctionAndRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n<|editable_region_end|>\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"上海今天的天气如何?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithReferencedRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t\tList<Document> documents = (List<Document>) response.getMetadata()\n\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\tAssertions.assertNotNull(documents);\n\n\t\tfor (Document document : documents) {\n\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t}\n\t}\n\n\t@Test\n\tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\n\t\t\tChatCompletionFinishReason finishReason = ChatCompletionFinishReason\n\t\t\t\t.valueOf(data.getResult().getMetadata().getFinishReason());\n\t\t\tif (finishReason != ChatCompletionFinishReason.NULL) {\n\t\t\t\tList<Document> documents = (List<Document>) data.getMetadata()\n\t\t\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\t\t\tAssertions.assertNotNull(documents);\n\n\t\t\t\tfor (Document document : documents) {\n\t\t\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t\t\t}\n\t\t\t}\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithMemory() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultAdvisors(new MessageChatMemoryAdvisor(new InMemoryChatMemory()))\n\t\t\t.build();\n\n\t\tString conversantId = UUID.randomUUID().toString();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\n\t\tresponse = chatClient.prompt()\n\t\t\t.user(\"可以给一些使用示例吗?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tcontent = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid reader() {\n\t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeApi, null);\n\t\tList<Document> documentList = reader.get();\n\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeApi);\n\t\tList<Document> transformerList = transformer.apply(documentList);\n\t\tSystem.out.println(transformerList.size());\n\t}\n\n\t@Test\n\tvoid embed() {\n\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeApi);\n\t\tDocument document = new Document(\"你好阿里云\");\n\t\tfloat[] vectorList = embeddingModel.embed(document);\n\t\tSystem.out.println(vectorList.length);\n\t}\n\n\t@Test\n\tvoid vectorStore() {\n\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n\t\tList<Document> documentList = Arrays.asList(\n\t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_f3cce7cab0b74e3d98a8d684f6fc4b55_10001\", \"abc\", new HashMap<>()));\n\t\t// cloudStore.add(documentList);\n\t\tcloudStore.delete(Arrays.asList(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\"));\n\t\tList<Document> documents = cloudStore.similaritySearch(\"南方电网\");\n\t\tSystem.out.println(documents.size());\n\t}\n\n}", "model_target_text": "\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionAndRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n", "region_before": "}\n\n\t@Test\n\tvoid callWithFunctionAndRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();", "jaccard_similarity": 0.4090909090909091, "semantic_cosine_similarity": 1.000000238418579, "llm_judge": {"label": "BAD", "confidence": 5, "reasons": "The target snippet is truncated and incomplete, ending abruptly inside the method. It also contains a variable name mismatch ('dashscopeChatApi' vs 'dashscopeApi') inconsistent with the input. The snippet does not fully cover the editable region and is syntactically invalid as is.", "flags": ["TRUNCATED", "MISMATCHED_REGION"]}}
{"commit_sha": "5840b7349a002a847c6c3070276e040b8381e0d0", "commit_short_sha": "5840b734", "commit_message": "fix: incrementalOutput not working (#17)\n\n1. set enable search to false for default chat configuration.\r\n2. Fix incremental output issue and support streaming function call.", "commit_author": "TianBing Ye", "commit_date": "2024-09-23T16:46:24+08:00", "total_files_in_commit": 8, "sequence_order": 7, "file_path": "spring-ai-alibaba-core/src/test/java/com/alibaba/cloud/ai/dashscope/chat/client/DashScopeChatClientIT.java", "file_extension": "java", "change_type": "modified", "lines_added": 27, "lines_deleted": 14, "total_changes": 41, "before_content": "/*\n * Copyright 2023-2024 the original author or authors.\n *\n * Licensed under the Apache License, Version 2.0 (the \"License\");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *      https://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an \"AS IS\" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\n\npackage com.alibaba.cloud.ai.dashscope.chat.client;\n\nimport com.alibaba.cloud.ai.advisor.DocumentRetrievalAdvisor;\nimport com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\nimport com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\nimport com.alibaba.cloud.ai.dashscope.embedding.DashScopeEmbeddingModel;\nimport com.alibaba.cloud.ai.dashscope.rag.*;\nimport org.junit.jupiter.api.Assertions;\nimport org.junit.jupiter.api.Test;\nimport org.slf4j.Logger;\nimport org.slf4j.LoggerFactory;\nimport org.springframework.ai.chat.client.ChatClient;\nimport org.springframework.ai.chat.client.advisor.MessageChatMemoryAdvisor;\nimport org.springframework.ai.chat.memory.InMemoryChatMemory;\nimport org.springframework.ai.chat.model.ChatResponse;\nimport org.springframework.ai.document.Document;\nimport org.springframework.ai.document.DocumentRetriever;\nimport org.springframework.beans.factory.annotation.Autowired;\nimport org.springframework.beans.factory.annotation.Value;\nimport org.springframework.boot.test.context.SpringBootTest;\nimport org.springframework.core.io.Resource;\nimport org.springframework.test.context.TestPropertySource;\nimport reactor.core.publisher.Flux;\n\nimport java.io.IOException;\nimport java.nio.charset.StandardCharsets;\nimport java.util.Arrays;\nimport java.util.HashMap;\nimport java.util.List;\nimport java.util.UUID;\nimport java.util.concurrent.CountDownLatch;\n\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_CONVERSATION_ID_KEY;\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_RETRIEVE_SIZE_KEY;\n\n/**\n * Title React agent test cases.<br/>\n * Description React agent test cases.<br/>\n * Created at 2024-08-19 10:15\n *\n * @author yuanci.ytb\n * @version 1.0.0\n * @since jdk8\n */\n\n@TestPropertySource(\"classpath:application.yml\")\n@SpringBootTest(classes = { DashScopeAutoConfiguration.class, DashScopeFunctionTestConfiguration.class,\n\t\tMockOrderService.class })\npublic class DashScopeChatClientIT {\n\n\tprivate static final Logger logger = LoggerFactory.getLogger(DashScopeChatClientIT.class);\n\n\t@Autowired\n\tprivate DashScopeChatModel dashscopeChatModel;\n\n\t@Autowired\n\tprivate DashScopeApi dashscopeApi;\n\n\t@Value(\"classpath:/prompts/rag/system-qa.st\")\n\tprivate Resource systemResource;\n\n\t@Value(\"classpath:/prompts/rag/system-qa-ref.st\")\n\tprivate Resource systemResourceRef;\n\n\t@Test\n\tvoid callTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"spring ai alibaba 是什么?\").call().chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithFunctionTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.function(\"getWeather\", \"根据城市查询天气\", new MockWeatherService())\n\t\t\t.user(\"杭州今天的天气如何?\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionBeanTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).defaultFunctions(\"getOrderFunction\").build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.functions(\"getCurrentWeather\")\n\t\t\t.user(\"帮我一下订单, 用户编号为1001, 订单编号为2001\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionAndRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"上海今天的天气如何?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithReferencedRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t\tList<Document> documents = (List<Document>) response.getMetadata()\n\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\tAssertions.assertNotNull(documents);\n\n\t\tfor (Document document : documents) {\n\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t}\n\t}\n\n\t@Test\n\tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\n\t\t\tChatCompletionFinishReason finishReason = ChatCompletionFinishReason\n\t\t\t\t.valueOf(data.getResult().getMetadata().getFinishReason());\n\t\t\tif (finishReason != ChatCompletionFinishReason.NULL) {\n\t\t\t\tList<Document> documents = (List<Document>) data.getMetadata()\n\t\t\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\t\t\tAssertions.assertNotNull(documents);\n\n\t\t\t\tfor (Document document : documents) {\n\t\t\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t\t\t}\n\t\t\t}\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithMemory() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultAdvisors(new MessageChatMemoryAdvisor(new InMemoryChatMemory()))\n\t\t\t.build();\n\n\t\tString conversantId = UUID.randomUUID().toString();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\n\t\tresponse = chatClient.prompt()\n\t\t\t.user(\"可以给一些使用示例吗?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tcontent = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid reader() {\n\t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeApi, null);\n\t\tList<Document> documentList = reader.get();\n\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeApi);\n\t\tList<Document> transformerList = transformer.apply(documentList);\n\t\tSystem.out.println(transformerList.size());\n\t}\n\n\t@Test\n\tvoid embed() {\n\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeApi);\n\t\tDocument document = new Document(\"你好阿里云\");\n\t\tfloat[] vectorList = embeddingModel.embed(document);\n\t\tSystem.out.println(vectorList.length);\n\t}\n\n\t@Test\n\tvoid vectorStore() {\n\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n\t\tList<Document> documentList = Arrays.asList(\n\t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_f3cce7cab0b74e3d98a8d684f6fc4b55_10001\", \"abc\", new HashMap<>()));\n\t\t// cloudStore.add(documentList);\n\t\tcloudStore.delete(Arrays.asList(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\"));\n\t\tList<Document> documents = cloudStore.similaritySearch(\"南方电网\");\n\t\tSystem.out.println(documents.size());\n\t}\n\n}", "after_content": "/*\n * Copyright 2023-2024 the original author or authors.\n *\n * Licensed under the Apache License, Version 2.0 (the \"License\");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *      https://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an \"AS IS\" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\n\npackage com.alibaba.cloud.ai.dashscope.chat.client;\n\nimport com.alibaba.cloud.ai.advisor.DocumentRetrievalAdvisor;\nimport com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatOptions;\nimport com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\nimport com.alibaba.cloud.ai.dashscope.embedding.DashScopeEmbeddingModel;\nimport com.alibaba.cloud.ai.dashscope.rag.*;\nimport org.junit.jupiter.api.Assertions;\nimport org.junit.jupiter.api.Test;\nimport org.slf4j.Logger;\nimport org.slf4j.LoggerFactory;\nimport org.springframework.ai.chat.client.ChatClient;\nimport org.springframework.ai.chat.client.advisor.MessageChatMemoryAdvisor;\nimport org.springframework.ai.chat.memory.InMemoryChatMemory;\nimport org.springframework.ai.chat.model.ChatResponse;\nimport org.springframework.ai.document.Document;\nimport org.springframework.ai.document.DocumentRetriever;\nimport org.springframework.beans.factory.annotation.Autowired;\nimport org.springframework.beans.factory.annotation.Value;\nimport org.springframework.boot.test.context.SpringBootTest;\nimport org.springframework.core.io.Resource;\nimport org.springframework.test.context.TestPropertySource;\nimport reactor.core.publisher.Flux;\n\nimport java.io.IOException;\nimport java.nio.charset.StandardCharsets;\nimport java.util.Arrays;\nimport java.util.HashMap;\nimport java.util.List;\nimport java.util.UUID;\nimport java.util.concurrent.CountDownLatch;\n\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_CONVERSATION_ID_KEY;\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_RETRIEVE_SIZE_KEY;\n\n/**\n * Title React agent test cases.<br/>\n * Description React agent test cases.<br/>\n * Created at 2024-08-19 10:15\n *\n * @author yuanci.ytb\n * @version 1.0.0\n * @since jdk8\n */\n\n@TestPropertySource(\"classpath:application.yml\")\n@SpringBootTest(classes = { DashScopeAutoConfiguration.class, DashScopeFunctionTestConfiguration.class,\n\t\tMockOrderService.class })\npublic class DashScopeChatClientIT {\n\n\tprivate static final Logger logger = LoggerFactory.getLogger(DashScopeChatClientIT.class);\n\n\t@Autowired\n\tprivate DashScopeChatModel dashscopeChatModel;\n\n\t@Autowired\n\tprivate DashScopeApi dashscopeChatApi;\n\n\t@Value(\"classpath:/prompts/rag/system-qa.st\")\n\tprivate Resource systemResource;\n\n\t@Value(\"classpath:/prompts/rag/system-qa-ref.st\")\n\tprivate Resource systemResourceRef;\n\n\t@Test\n\tvoid callTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"spring ai alibaba 是什么?\").call().chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt()\n\t\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t\t.options(DashScopeChatOptions.builder()\n\t\t\t\t\t\t.withIncrementalOutput(true)\n\t\t\t\t\t\t.build())\n\t\t\t\t.stream()\n\t\t\t\t.chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithFunctionTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.function(\"getWeather\", \"根据城市查询天气\", new MockWeatherService())\n\t\t\t.user(\"杭州今天的天气如何?\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionBeanTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).defaultFunctions(\"getOrderFunction\").build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.functions(\"getCurrentWeather\")\n\t\t\t.user(\"帮我一下订单, 用户编号为1001, 订单编号为2001\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionAndRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt()\n\t\t\t\t.user(\"上海今天的天气如何?\")\n\t\t\t\t.options(DashScopeChatOptions.builder()\n\t\t\t\t\t\t.withIncrementalOutput(true)\n\t\t\t\t\t\t.build())\n\t\t\t\t.stream()\n\t\t\t\t.chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithReferencedRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t\tList<Document> documents = (List<Document>) response.getMetadata()\n\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\tAssertions.assertNotNull(documents);\n\n\t\tfor (Document document : documents) {\n\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t}\n\t}\n\n\t@Test\n\tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\n\t\t\tChatCompletionFinishReason finishReason = ChatCompletionFinishReason\n\t\t\t\t.valueOf(data.getResult().getMetadata().getFinishReason());\n\t\t\tif (finishReason != ChatCompletionFinishReason.NULL) {\n\t\t\t\tList<Document> documents = (List<Document>) data.getMetadata()\n\t\t\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\t\t\tAssertions.assertNotNull(documents);\n\n\t\t\t\tfor (Document document : documents) {\n\t\t\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t\t\t}\n\t\t\t}\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithMemory() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultAdvisors(new MessageChatMemoryAdvisor(new InMemoryChatMemory()))\n\t\t\t.build();\n\n\t\tString conversantId = UUID.randomUUID().toString();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\n\t\tresponse = chatClient.prompt()\n\t\t\t.user(\"可以给一些使用示例吗?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tcontent = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid reader() {\n\t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeChatApi, null);\n\t\tList<Document> documentList = reader.get();\n\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeChatApi);\n\t\tList<Document> transformerList = transformer.apply(documentList);\n\t\tSystem.out.println(transformerList.size());\n\t}\n\n\t@Test\n\tvoid embed() {\n\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeChatApi);\n\t\tDocument document = new Document(\"你好阿里云\");\n\t\tfloat[] vectorList = embeddingModel.embed(document);\n\t\tSystem.out.println(vectorList.length);\n\t}\n\n\t@Test\n\tvoid vectorStore() {\n\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeChatApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n\t\tList<Document> documentList = Arrays.asList(\n\t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_f3cce7cab0b74e3d98a8d684f6fc4b55_10001\", \"abc\", new HashMap<>()));\n\t\t// cloudStore.add(documentList);\n\t\tcloudStore.delete(Arrays.asList(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\"));\n\t\tList<Document> documents = cloudStore.similaritySearch(\"南方电网\");\n\t\tSystem.out.println(documents.size());\n\t}\n\n}", "patch": "@@ -21,6 +21,7 @@ import com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\n import com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\n import com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\n import com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\n+import com.alibaba.cloud.ai.dashscope.chat.DashScopeChatOptions;\n import com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\n import com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\n import com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\n@@ -75,7 +76,7 @@ public class DashScopeChatClientIT {\n \tprivate DashScopeChatModel dashscopeChatModel;\n \n \t@Autowired\n-\tprivate DashScopeApi dashscopeApi;\n+\tprivate DashScopeApi dashscopeChatApi;\n \n \t@Value(\"classpath:/prompts/rag/system-qa.st\")\n \tprivate Resource systemResource;\n@@ -85,7 +86,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callTest() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -102,14 +103,20 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid streamTest() throws InterruptedException, IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n \t\t\t.defaultAdvisors(\n \t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n \t\t\t.build();\n \n-\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n+\t\tFlux<ChatResponse> response = chatClient.prompt()\n+\t\t\t\t.user(\"如何快速开始百炼?\")\n+\t\t\t\t.options(DashScopeChatOptions.builder()\n+\t\t\t\t\t\t.withIncrementalOutput(true)\n+\t\t\t\t\t\t.build())\n+\t\t\t\t.stream()\n+\t\t\t\t.chatResponse();\n \n \t\tCountDownLatch cdl = new CountDownLatch(1);\n \t\tresponse.subscribe(data -> {\n@@ -159,7 +166,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callWithFunctionAndRagTest() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -178,7 +185,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -187,7 +194,13 @@ public class DashScopeChatClientIT {\n \t\t\t.defaultFunctions(\"weatherFunction\")\n \t\t\t.build();\n \n-\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"上海今天的天气如何?\").stream().chatResponse();\n+\t\tFlux<ChatResponse> response = chatClient.prompt()\n+\t\t\t\t.user(\"上海今天的天气如何?\")\n+\t\t\t\t.options(DashScopeChatOptions.builder()\n+\t\t\t\t\t\t.withIncrementalOutput(true)\n+\t\t\t\t\t\t.build())\n+\t\t\t\t.stream()\n+\t\t\t\t.chatResponse();\n \n \t\tCountDownLatch cdl = new CountDownLatch(1);\n \t\tresponse.subscribe(data -> {\n@@ -206,7 +219,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callWithReferencedRagTest() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -232,7 +245,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -272,7 +285,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callWithMemory() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -309,16 +322,16 @@ public class DashScopeChatClientIT {\n \t@Test\n \tvoid reader() {\n \t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n-\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeApi, null);\n+\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeChatApi, null);\n \t\tList<Document> documentList = reader.get();\n-\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeApi);\n+\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeChatApi);\n \t\tList<Document> transformerList = transformer.apply(documentList);\n \t\tSystem.out.println(transformerList.size());\n \t}\n \n \t@Test\n \tvoid embed() {\n-\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeApi);\n+\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeChatApi);\n \t\tDocument document = new Document(\"你好阿里云\");\n \t\tfloat[] vectorList = embeddingModel.embed(document);\n \t\tSystem.out.println(vectorList.length);\n@@ -326,7 +339,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid vectorStore() {\n-\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n+\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeChatApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n \t\tList<Document> documentList = Arrays.asList(\n \t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n \t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n", "editable_region_start": 177, "editable_region_end": 199, "change_labels": null, "chunk_id": 3, "total_chunks_in_file": 8, "is_multi_chunk_file": true, "model_input_text": "/*\n * Copyright 2023-2024 the original author or authors.\n *\n * Licensed under the Apache License, Version 2.0 (the \"License\");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *      https://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an \"AS IS\" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\n\npackage com.alibaba.cloud.ai.dashscope.chat.client;\n\nimport com.alibaba.cloud.ai.advisor.DocumentRetrievalAdvisor;\nimport com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\nimport com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\nimport com.alibaba.cloud.ai.dashscope.embedding.DashScopeEmbeddingModel;\nimport com.alibaba.cloud.ai.dashscope.rag.*;\nimport org.junit.jupiter.api.Assertions;\nimport org.junit.jupiter.api.Test;\nimport org.slf4j.Logger;\nimport org.slf4j.LoggerFactory;\nimport org.springframework.ai.chat.client.ChatClient;\nimport org.springframework.ai.chat.client.advisor.MessageChatMemoryAdvisor;\nimport org.springframework.ai.chat.memory.InMemoryChatMemory;\nimport org.springframework.ai.chat.model.ChatResponse;\nimport org.springframework.ai.document.Document;\nimport org.springframework.ai.document.DocumentRetriever;\nimport org.springframework.beans.factory.annotation.Autowired;\nimport org.springframework.beans.factory.annotation.Value;\nimport org.springframework.boot.test.context.SpringBootTest;\nimport org.springframework.core.io.Resource;\nimport org.springframework.test.context.TestPropertySource;\nimport reactor.core.publisher.Flux;\n\nimport java.io.IOException;\nimport java.nio.charset.StandardCharsets;\nimport java.util.Arrays;\nimport java.util.HashMap;\nimport java.util.List;\nimport java.util.UUID;\nimport java.util.concurrent.CountDownLatch;\n\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_CONVERSATION_ID_KEY;\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_RETRIEVE_SIZE_KEY;\n\n/**\n * Title React agent test cases.<br/>\n * Description React agent test cases.<br/>\n * Created at 2024-08-19 10:15\n *\n * @author yuanci.ytb\n * @version 1.0.0\n * @since jdk8\n */\n\n@TestPropertySource(\"classpath:application.yml\")\n@SpringBootTest(classes = { DashScopeAutoConfiguration.class, DashScopeFunctionTestConfiguration.class,\n\t\tMockOrderService.class })\npublic class DashScopeChatClientIT {\n\n\tprivate static final Logger logger = LoggerFactory.getLogger(DashScopeChatClientIT.class);\n\n\t@Autowired\n\tprivate DashScopeChatModel dashscopeChatModel;\n\n\t@Autowired\n\tprivate DashScopeApi dashscopeApi;\n\n\t@Value(\"classpath:/prompts/rag/system-qa.st\")\n\tprivate Resource systemResource;\n\n\t@Value(\"classpath:/prompts/rag/system-qa-ref.st\")\n\tprivate Resource systemResourceRef;\n\n\t@Test\n\tvoid callTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"spring ai alibaba 是什么?\").call().chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithFunctionTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.function(\"getWeather\", \"根据城市查询天气\", new MockWeatherService())\n\t\t\t.user(\"杭州今天的天气如何?\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionBeanTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).defaultFunctions(\"getOrderFunction\").build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.functions(\"getCurrentWeather\")\n\t\t\t.user(\"帮我一下订单, 用户编号为1001, 订单编号为2001\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionAndRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n<|editable_region_start|>\n\t}\n\n\t@Test\n\tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"上海今天的天气如何?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n<|editable_region_end|>\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithReferencedRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t\tList<Document> documents = (List<Document>) response.getMetadata()\n\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\tAssertions.assertNotNull(documents);\n\n\t\tfor (Document document : documents) {\n\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t}\n\t}\n\n\t@Test\n\tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\n\t\t\tChatCompletionFinishReason finishReason = ChatCompletionFinishReason\n\t\t\t\t.valueOf(data.getResult().getMetadata().getFinishReason());\n\t\t\tif (finishReason != ChatCompletionFinishReason.NULL) {\n\t\t\t\tList<Document> documents = (List<Document>) data.getMetadata()\n\t\t\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\t\t\tAssertions.assertNotNull(documents);\n\n\t\t\t\tfor (Document document : documents) {\n\t\t\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t\t\t}\n\t\t\t}\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithMemory() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultAdvisors(new MessageChatMemoryAdvisor(new InMemoryChatMemory()))\n\t\t\t.build();\n\n\t\tString conversantId = UUID.randomUUID().toString();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\n\t\tresponse = chatClient.prompt()\n\t\t\t.user(\"可以给一些使用示例吗?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tcontent = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid reader() {\n\t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeApi, null);\n\t\tList<Document> documentList = reader.get();\n\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeApi);\n\t\tList<Document> transformerList = transformer.apply(documentList);\n\t\tSystem.out.println(transformerList.size());\n\t}\n\n\t@Test\n\tvoid embed() {\n\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeApi);\n\t\tDocument document = new Document(\"你好阿里云\");\n\t\tfloat[] vectorList = embeddingModel.embed(document);\n\t\tSystem.out.println(vectorList.length);\n\t}\n\n\t@Test\n\tvoid vectorStore() {\n\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n\t\tList<Document> documentList = Arrays.asList(\n\t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_f3cce7cab0b74e3d98a8d684f6fc4b55_10001\", \"abc\", new HashMap<>()));\n\t\t// cloudStore.add(documentList);\n\t\tcloudStore.delete(Arrays.asList(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\"));\n\t\tList<Document> documents = cloudStore.similaritySearch(\"南方电网\");\n\t\tSystem.out.println(documents.size());\n\t}\n\n}", "model_target_text": "\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt()\n\t\t\t\t.user(\"上海今天的天气如何?\")\n\t\t\t\t.options(DashScopeChatOptions.builder()", "region_before": "}\n\n\t@Test\n\tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"上海今天的天气如何?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();", "jaccard_similarity": 0.59375, "semantic_cosine_similarity": 1.0, "llm_judge": {"label": "BAD", "confidence": 5, "reasons": "The target snippet is truncated and incomplete, ending abruptly in the middle of a method call chain. It also contains a variable name 'dashscopeChatApi' that does not match the input context's 'dashscopeApi', indicating inconsistency. The target does not align with the editable region boundaries and is not a reasonable completion of the input.", "flags": ["TRUNCATED", "MISMATCHED_REGION", "MINOR_NOISE"]}}
{"commit_sha": "5840b7349a002a847c6c3070276e040b8381e0d0", "commit_short_sha": "5840b734", "commit_message": "fix: incrementalOutput not working (#17)\n\n1. set enable search to false for default chat configuration.\r\n2. Fix incremental output issue and support streaming function call.", "commit_author": "TianBing Ye", "commit_date": "2024-09-23T16:46:24+08:00", "total_files_in_commit": 8, "sequence_order": 7, "file_path": "spring-ai-alibaba-core/src/test/java/com/alibaba/cloud/ai/dashscope/chat/client/DashScopeChatClientIT.java", "file_extension": "java", "change_type": "modified", "lines_added": 27, "lines_deleted": 14, "total_changes": 41, "before_content": "/*\n * Copyright 2023-2024 the original author or authors.\n *\n * Licensed under the Apache License, Version 2.0 (the \"License\");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *      https://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an \"AS IS\" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\n\npackage com.alibaba.cloud.ai.dashscope.chat.client;\n\nimport com.alibaba.cloud.ai.advisor.DocumentRetrievalAdvisor;\nimport com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\nimport com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\nimport com.alibaba.cloud.ai.dashscope.embedding.DashScopeEmbeddingModel;\nimport com.alibaba.cloud.ai.dashscope.rag.*;\nimport org.junit.jupiter.api.Assertions;\nimport org.junit.jupiter.api.Test;\nimport org.slf4j.Logger;\nimport org.slf4j.LoggerFactory;\nimport org.springframework.ai.chat.client.ChatClient;\nimport org.springframework.ai.chat.client.advisor.MessageChatMemoryAdvisor;\nimport org.springframework.ai.chat.memory.InMemoryChatMemory;\nimport org.springframework.ai.chat.model.ChatResponse;\nimport org.springframework.ai.document.Document;\nimport org.springframework.ai.document.DocumentRetriever;\nimport org.springframework.beans.factory.annotation.Autowired;\nimport org.springframework.beans.factory.annotation.Value;\nimport org.springframework.boot.test.context.SpringBootTest;\nimport org.springframework.core.io.Resource;\nimport org.springframework.test.context.TestPropertySource;\nimport reactor.core.publisher.Flux;\n\nimport java.io.IOException;\nimport java.nio.charset.StandardCharsets;\nimport java.util.Arrays;\nimport java.util.HashMap;\nimport java.util.List;\nimport java.util.UUID;\nimport java.util.concurrent.CountDownLatch;\n\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_CONVERSATION_ID_KEY;\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_RETRIEVE_SIZE_KEY;\n\n/**\n * Title React agent test cases.<br/>\n * Description React agent test cases.<br/>\n * Created at 2024-08-19 10:15\n *\n * @author yuanci.ytb\n * @version 1.0.0\n * @since jdk8\n */\n\n@TestPropertySource(\"classpath:application.yml\")\n@SpringBootTest(classes = { DashScopeAutoConfiguration.class, DashScopeFunctionTestConfiguration.class,\n\t\tMockOrderService.class })\npublic class DashScopeChatClientIT {\n\n\tprivate static final Logger logger = LoggerFactory.getLogger(DashScopeChatClientIT.class);\n\n\t@Autowired\n\tprivate DashScopeChatModel dashscopeChatModel;\n\n\t@Autowired\n\tprivate DashScopeApi dashscopeApi;\n\n\t@Value(\"classpath:/prompts/rag/system-qa.st\")\n\tprivate Resource systemResource;\n\n\t@Value(\"classpath:/prompts/rag/system-qa-ref.st\")\n\tprivate Resource systemResourceRef;\n\n\t@Test\n\tvoid callTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"spring ai alibaba 是什么?\").call().chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithFunctionTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.function(\"getWeather\", \"根据城市查询天气\", new MockWeatherService())\n\t\t\t.user(\"杭州今天的天气如何?\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionBeanTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).defaultFunctions(\"getOrderFunction\").build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.functions(\"getCurrentWeather\")\n\t\t\t.user(\"帮我一下订单, 用户编号为1001, 订单编号为2001\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionAndRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"上海今天的天气如何?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithReferencedRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t\tList<Document> documents = (List<Document>) response.getMetadata()\n\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\tAssertions.assertNotNull(documents);\n\n\t\tfor (Document document : documents) {\n\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t}\n\t}\n\n\t@Test\n\tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\n\t\t\tChatCompletionFinishReason finishReason = ChatCompletionFinishReason\n\t\t\t\t.valueOf(data.getResult().getMetadata().getFinishReason());\n\t\t\tif (finishReason != ChatCompletionFinishReason.NULL) {\n\t\t\t\tList<Document> documents = (List<Document>) data.getMetadata()\n\t\t\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\t\t\tAssertions.assertNotNull(documents);\n\n\t\t\t\tfor (Document document : documents) {\n\t\t\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t\t\t}\n\t\t\t}\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithMemory() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultAdvisors(new MessageChatMemoryAdvisor(new InMemoryChatMemory()))\n\t\t\t.build();\n\n\t\tString conversantId = UUID.randomUUID().toString();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\n\t\tresponse = chatClient.prompt()\n\t\t\t.user(\"可以给一些使用示例吗?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tcontent = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid reader() {\n\t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeApi, null);\n\t\tList<Document> documentList = reader.get();\n\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeApi);\n\t\tList<Document> transformerList = transformer.apply(documentList);\n\t\tSystem.out.println(transformerList.size());\n\t}\n\n\t@Test\n\tvoid embed() {\n\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeApi);\n\t\tDocument document = new Document(\"你好阿里云\");\n\t\tfloat[] vectorList = embeddingModel.embed(document);\n\t\tSystem.out.println(vectorList.length);\n\t}\n\n\t@Test\n\tvoid vectorStore() {\n\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n\t\tList<Document> documentList = Arrays.asList(\n\t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_f3cce7cab0b74e3d98a8d684f6fc4b55_10001\", \"abc\", new HashMap<>()));\n\t\t// cloudStore.add(documentList);\n\t\tcloudStore.delete(Arrays.asList(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\"));\n\t\tList<Document> documents = cloudStore.similaritySearch(\"南方电网\");\n\t\tSystem.out.println(documents.size());\n\t}\n\n}", "after_content": "/*\n * Copyright 2023-2024 the original author or authors.\n *\n * Licensed under the Apache License, Version 2.0 (the \"License\");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *      https://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an \"AS IS\" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\n\npackage com.alibaba.cloud.ai.dashscope.chat.client;\n\nimport com.alibaba.cloud.ai.advisor.DocumentRetrievalAdvisor;\nimport com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatOptions;\nimport com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\nimport com.alibaba.cloud.ai.dashscope.embedding.DashScopeEmbeddingModel;\nimport com.alibaba.cloud.ai.dashscope.rag.*;\nimport org.junit.jupiter.api.Assertions;\nimport org.junit.jupiter.api.Test;\nimport org.slf4j.Logger;\nimport org.slf4j.LoggerFactory;\nimport org.springframework.ai.chat.client.ChatClient;\nimport org.springframework.ai.chat.client.advisor.MessageChatMemoryAdvisor;\nimport org.springframework.ai.chat.memory.InMemoryChatMemory;\nimport org.springframework.ai.chat.model.ChatResponse;\nimport org.springframework.ai.document.Document;\nimport org.springframework.ai.document.DocumentRetriever;\nimport org.springframework.beans.factory.annotation.Autowired;\nimport org.springframework.beans.factory.annotation.Value;\nimport org.springframework.boot.test.context.SpringBootTest;\nimport org.springframework.core.io.Resource;\nimport org.springframework.test.context.TestPropertySource;\nimport reactor.core.publisher.Flux;\n\nimport java.io.IOException;\nimport java.nio.charset.StandardCharsets;\nimport java.util.Arrays;\nimport java.util.HashMap;\nimport java.util.List;\nimport java.util.UUID;\nimport java.util.concurrent.CountDownLatch;\n\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_CONVERSATION_ID_KEY;\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_RETRIEVE_SIZE_KEY;\n\n/**\n * Title React agent test cases.<br/>\n * Description React agent test cases.<br/>\n * Created at 2024-08-19 10:15\n *\n * @author yuanci.ytb\n * @version 1.0.0\n * @since jdk8\n */\n\n@TestPropertySource(\"classpath:application.yml\")\n@SpringBootTest(classes = { DashScopeAutoConfiguration.class, DashScopeFunctionTestConfiguration.class,\n\t\tMockOrderService.class })\npublic class DashScopeChatClientIT {\n\n\tprivate static final Logger logger = LoggerFactory.getLogger(DashScopeChatClientIT.class);\n\n\t@Autowired\n\tprivate DashScopeChatModel dashscopeChatModel;\n\n\t@Autowired\n\tprivate DashScopeApi dashscopeChatApi;\n\n\t@Value(\"classpath:/prompts/rag/system-qa.st\")\n\tprivate Resource systemResource;\n\n\t@Value(\"classpath:/prompts/rag/system-qa-ref.st\")\n\tprivate Resource systemResourceRef;\n\n\t@Test\n\tvoid callTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"spring ai alibaba 是什么?\").call().chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt()\n\t\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t\t.options(DashScopeChatOptions.builder()\n\t\t\t\t\t\t.withIncrementalOutput(true)\n\t\t\t\t\t\t.build())\n\t\t\t\t.stream()\n\t\t\t\t.chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithFunctionTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.function(\"getWeather\", \"根据城市查询天气\", new MockWeatherService())\n\t\t\t.user(\"杭州今天的天气如何?\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionBeanTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).defaultFunctions(\"getOrderFunction\").build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.functions(\"getCurrentWeather\")\n\t\t\t.user(\"帮我一下订单, 用户编号为1001, 订单编号为2001\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionAndRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt()\n\t\t\t\t.user(\"上海今天的天气如何?\")\n\t\t\t\t.options(DashScopeChatOptions.builder()\n\t\t\t\t\t\t.withIncrementalOutput(true)\n\t\t\t\t\t\t.build())\n\t\t\t\t.stream()\n\t\t\t\t.chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithReferencedRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t\tList<Document> documents = (List<Document>) response.getMetadata()\n\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\tAssertions.assertNotNull(documents);\n\n\t\tfor (Document document : documents) {\n\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t}\n\t}\n\n\t@Test\n\tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\n\t\t\tChatCompletionFinishReason finishReason = ChatCompletionFinishReason\n\t\t\t\t.valueOf(data.getResult().getMetadata().getFinishReason());\n\t\t\tif (finishReason != ChatCompletionFinishReason.NULL) {\n\t\t\t\tList<Document> documents = (List<Document>) data.getMetadata()\n\t\t\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\t\t\tAssertions.assertNotNull(documents);\n\n\t\t\t\tfor (Document document : documents) {\n\t\t\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t\t\t}\n\t\t\t}\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithMemory() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultAdvisors(new MessageChatMemoryAdvisor(new InMemoryChatMemory()))\n\t\t\t.build();\n\n\t\tString conversantId = UUID.randomUUID().toString();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\n\t\tresponse = chatClient.prompt()\n\t\t\t.user(\"可以给一些使用示例吗?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tcontent = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid reader() {\n\t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeChatApi, null);\n\t\tList<Document> documentList = reader.get();\n\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeChatApi);\n\t\tList<Document> transformerList = transformer.apply(documentList);\n\t\tSystem.out.println(transformerList.size());\n\t}\n\n\t@Test\n\tvoid embed() {\n\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeChatApi);\n\t\tDocument document = new Document(\"你好阿里云\");\n\t\tfloat[] vectorList = embeddingModel.embed(document);\n\t\tSystem.out.println(vectorList.length);\n\t}\n\n\t@Test\n\tvoid vectorStore() {\n\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeChatApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n\t\tList<Document> documentList = Arrays.asList(\n\t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_f3cce7cab0b74e3d98a8d684f6fc4b55_10001\", \"abc\", new HashMap<>()));\n\t\t// cloudStore.add(documentList);\n\t\tcloudStore.delete(Arrays.asList(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\"));\n\t\tList<Document> documents = cloudStore.similaritySearch(\"南方电网\");\n\t\tSystem.out.println(documents.size());\n\t}\n\n}", "patch": "@@ -21,6 +21,7 @@ import com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\n import com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\n import com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\n import com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\n+import com.alibaba.cloud.ai.dashscope.chat.DashScopeChatOptions;\n import com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\n import com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\n import com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\n@@ -75,7 +76,7 @@ public class DashScopeChatClientIT {\n \tprivate DashScopeChatModel dashscopeChatModel;\n \n \t@Autowired\n-\tprivate DashScopeApi dashscopeApi;\n+\tprivate DashScopeApi dashscopeChatApi;\n \n \t@Value(\"classpath:/prompts/rag/system-qa.st\")\n \tprivate Resource systemResource;\n@@ -85,7 +86,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callTest() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -102,14 +103,20 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid streamTest() throws InterruptedException, IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n \t\t\t.defaultAdvisors(\n \t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n \t\t\t.build();\n \n-\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n+\t\tFlux<ChatResponse> response = chatClient.prompt()\n+\t\t\t\t.user(\"如何快速开始百炼?\")\n+\t\t\t\t.options(DashScopeChatOptions.builder()\n+\t\t\t\t\t\t.withIncrementalOutput(true)\n+\t\t\t\t\t\t.build())\n+\t\t\t\t.stream()\n+\t\t\t\t.chatResponse();\n \n \t\tCountDownLatch cdl = new CountDownLatch(1);\n \t\tresponse.subscribe(data -> {\n@@ -159,7 +166,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callWithFunctionAndRagTest() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -178,7 +185,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -187,7 +194,13 @@ public class DashScopeChatClientIT {\n \t\t\t.defaultFunctions(\"weatherFunction\")\n \t\t\t.build();\n \n-\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"上海今天的天气如何?\").stream().chatResponse();\n+\t\tFlux<ChatResponse> response = chatClient.prompt()\n+\t\t\t\t.user(\"上海今天的天气如何?\")\n+\t\t\t\t.options(DashScopeChatOptions.builder()\n+\t\t\t\t\t\t.withIncrementalOutput(true)\n+\t\t\t\t\t\t.build())\n+\t\t\t\t.stream()\n+\t\t\t\t.chatResponse();\n \n \t\tCountDownLatch cdl = new CountDownLatch(1);\n \t\tresponse.subscribe(data -> {\n@@ -206,7 +219,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callWithReferencedRagTest() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -232,7 +245,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -272,7 +285,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid callWithMemory() throws IOException {\n-\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n+\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeChatApi,\n \t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n \n \t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n@@ -309,16 +322,16 @@ public class DashScopeChatClientIT {\n \t@Test\n \tvoid reader() {\n \t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n-\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeApi, null);\n+\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeChatApi, null);\n \t\tList<Document> documentList = reader.get();\n-\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeApi);\n+\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeChatApi);\n \t\tList<Document> transformerList = transformer.apply(documentList);\n \t\tSystem.out.println(transformerList.size());\n \t}\n \n \t@Test\n \tvoid embed() {\n-\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeApi);\n+\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeChatApi);\n \t\tDocument document = new Document(\"你好阿里云\");\n \t\tfloat[] vectorList = embeddingModel.embed(document);\n \t\tSystem.out.println(vectorList.length);\n@@ -326,7 +339,7 @@ public class DashScopeChatClientIT {\n \n \t@Test\n \tvoid vectorStore() {\n-\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n+\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeChatApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n \t\tList<Document> documentList = Arrays.asList(\n \t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n \t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n", "editable_region_start": 205, "editable_region_end": 218, "change_labels": null, "chunk_id": 4, "total_chunks_in_file": 8, "is_multi_chunk_file": true, "model_input_text": "/*\n * Copyright 2023-2024 the original author or authors.\n *\n * Licensed under the Apache License, Version 2.0 (the \"License\");\n * you may not use this file except in compliance with the License.\n * You may obtain a copy of the License at\n *\n *      https://www.apache.org/licenses/LICENSE-2.0\n *\n * Unless required by applicable law or agreed to in writing, software\n * distributed under the License is distributed on an \"AS IS\" BASIS,\n * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n * See the License for the specific language governing permissions and\n * limitations under the License.\n */\n\npackage com.alibaba.cloud.ai.dashscope.chat.client;\n\nimport com.alibaba.cloud.ai.advisor.DocumentRetrievalAdvisor;\nimport com.alibaba.cloud.ai.autoconfigure.dashscope.DashScopeAutoConfiguration;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi;\nimport com.alibaba.cloud.ai.dashscope.api.DashScopeApi.ChatCompletionFinishReason;\nimport com.alibaba.cloud.ai.dashscope.chat.DashScopeChatModel;\nimport com.alibaba.cloud.ai.dashscope.tool.DashScopeFunctionTestConfiguration;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockOrderService;\nimport com.alibaba.cloud.ai.dashscope.chat.tool.MockWeatherService;\nimport com.alibaba.cloud.ai.dashscope.embedding.DashScopeEmbeddingModel;\nimport com.alibaba.cloud.ai.dashscope.rag.*;\nimport org.junit.jupiter.api.Assertions;\nimport org.junit.jupiter.api.Test;\nimport org.slf4j.Logger;\nimport org.slf4j.LoggerFactory;\nimport org.springframework.ai.chat.client.ChatClient;\nimport org.springframework.ai.chat.client.advisor.MessageChatMemoryAdvisor;\nimport org.springframework.ai.chat.memory.InMemoryChatMemory;\nimport org.springframework.ai.chat.model.ChatResponse;\nimport org.springframework.ai.document.Document;\nimport org.springframework.ai.document.DocumentRetriever;\nimport org.springframework.beans.factory.annotation.Autowired;\nimport org.springframework.beans.factory.annotation.Value;\nimport org.springframework.boot.test.context.SpringBootTest;\nimport org.springframework.core.io.Resource;\nimport org.springframework.test.context.TestPropertySource;\nimport reactor.core.publisher.Flux;\n\nimport java.io.IOException;\nimport java.nio.charset.StandardCharsets;\nimport java.util.Arrays;\nimport java.util.HashMap;\nimport java.util.List;\nimport java.util.UUID;\nimport java.util.concurrent.CountDownLatch;\n\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_CONVERSATION_ID_KEY;\nimport static org.springframework.ai.chat.client.advisor.AbstractChatMemoryAdvisor.CHAT_MEMORY_RETRIEVE_SIZE_KEY;\n\n/**\n * Title React agent test cases.<br/>\n * Description React agent test cases.<br/>\n * Created at 2024-08-19 10:15\n *\n * @author yuanci.ytb\n * @version 1.0.0\n * @since jdk8\n */\n\n@TestPropertySource(\"classpath:application.yml\")\n@SpringBootTest(classes = { DashScopeAutoConfiguration.class, DashScopeFunctionTestConfiguration.class,\n\t\tMockOrderService.class })\npublic class DashScopeChatClientIT {\n\n\tprivate static final Logger logger = LoggerFactory.getLogger(DashScopeChatClientIT.class);\n\n\t@Autowired\n\tprivate DashScopeChatModel dashscopeChatModel;\n\n\t@Autowired\n\tprivate DashScopeApi dashscopeApi;\n\n\t@Value(\"classpath:/prompts/rag/system-qa.st\")\n\tprivate Resource systemResource;\n\n\t@Value(\"classpath:/prompts/rag/system-qa-ref.st\")\n\tprivate Resource systemResourceRef;\n\n\t@Test\n\tvoid callTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"spring ai alibaba 是什么?\").call().chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithFunctionTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.function(\"getWeather\", \"根据城市查询天气\", new MockWeatherService())\n\t\t\t.user(\"杭州今天的天气如何?\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionBeanTest() {\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel).defaultFunctions(\"getOrderFunction\").build();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.functions(\"getCurrentWeather\")\n\t\t\t.user(\"帮我一下订单, 用户编号为1001, 订单编号为2001\")\n\t\t\t.call()\n\t\t\t.chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid callWithFunctionAndRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid streamCallWithFunctionAndRagTest() throws InterruptedException, IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultFunctions(\"weatherFunction\")\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"上海今天的天气如何?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n<|editable_region_start|>\n\t}\n\n\t@Test\n\tvoid callWithReferencedRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();\n\n<|editable_region_end|>\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t\tList<Document> documents = (List<Document>) response.getMetadata()\n\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\tAssertions.assertNotNull(documents);\n\n\t\tfor (Document document : documents) {\n\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t}\n\t}\n\n\t@Test\n\tvoid streamCallWithReferencedRagTest() throws IOException, InterruptedException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tFlux<ChatResponse> response = chatClient.prompt().user(\"如何快速开始百炼?\").stream().chatResponse();\n\n\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\n\t\t\tChatCompletionFinishReason finishReason = ChatCompletionFinishReason\n\t\t\t\t.valueOf(data.getResult().getMetadata().getFinishReason());\n\t\t\tif (finishReason != ChatCompletionFinishReason.NULL) {\n\t\t\t\tList<Document> documents = (List<Document>) data.getMetadata()\n\t\t\t\t\t.get(DashScopeDocumentRetrievalAdvisor.RETRIEVED_DOCUMENTS);\n\t\t\t\tAssertions.assertNotNull(documents);\n\n\t\t\t\tfor (Document document : documents) {\n\t\t\t\t\tlogger.info(\"referenced doc name: {}, title: {}, score: {}\", document.getMetadata().get(\"doc_name\"),\n\t\t\t\t\t\t\tdocument.getMetadata().get(\"title\"), document.getMetadata().get(\"_score\"));\n\t\t\t\t}\n\t\t\t}\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}\n\n\t@Test\n\tvoid callWithMemory() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(\n\t\t\t\t\tnew DocumentRetrievalAdvisor(retriever, systemResource.getContentAsString(StandardCharsets.UTF_8)))\n\t\t\t.defaultAdvisors(new MessageChatMemoryAdvisor(new InMemoryChatMemory()))\n\t\t\t.build();\n\n\t\tString conversantId = UUID.randomUUID().toString();\n\n\t\tChatResponse response = chatClient.prompt()\n\t\t\t.user(\"如何快速开始百炼?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tString content = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\n\t\tresponse = chatClient.prompt()\n\t\t\t.user(\"可以给一些使用示例吗?\")\n\t\t\t.advisors(spec -> spec.param(CHAT_MEMORY_CONVERSATION_ID_KEY, conversantId)\n\t\t\t\t.param(CHAT_MEMORY_RETRIEVE_SIZE_KEY, 10))\n\t\t\t.call()\n\t\t\t.chatResponse();\n\t\tcontent = response.getResult().getOutput().getContent();\n\t\tAssertions.assertNotNull(content);\n\n\t\tlogger.info(\"content: {}\", content);\n\t}\n\n\t@Test\n\tvoid reader() {\n\t\tString filePath = \"/Users/nuocheng.lxm/Desktop/新能源产业有哪些-36氪.pdf\";\n\t\tDashScopeDocumentCloudReader reader = new DashScopeDocumentCloudReader(filePath, dashscopeApi, null);\n\t\tList<Document> documentList = reader.get();\n\t\tDashScopeDocumentTransformer transformer = new DashScopeDocumentTransformer(dashscopeApi);\n\t\tList<Document> transformerList = transformer.apply(documentList);\n\t\tSystem.out.println(transformerList.size());\n\t}\n\n\t@Test\n\tvoid embed() {\n\t\tDashScopeEmbeddingModel embeddingModel = new DashScopeEmbeddingModel(dashscopeApi);\n\t\tDocument document = new Document(\"你好阿里云\");\n\t\tfloat[] vectorList = embeddingModel.embed(document);\n\t\tSystem.out.println(vectorList.length);\n\t}\n\n\t@Test\n\tvoid vectorStore() {\n\t\tDashScopeCloudStore cloudStore = new DashScopeCloudStore(dashscopeApi, new DashScopeStoreOptions(\"诺成SpringAI\"));\n\t\tList<Document> documentList = Arrays.asList(\n\t\t\t\tnew Document(\"file_f0b6b18b14994ed8a0b45648ce5d0da5_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\", \"abc\", new HashMap<>()),\n\t\t\t\tnew Document(\"file_f3cce7cab0b74e3d98a8d684f6fc4b55_10001\", \"abc\", new HashMap<>()));\n\t\t// cloudStore.add(documentList);\n\t\tcloudStore.delete(Arrays.asList(\"file_d3083d64026d4864b4558d18f9ca2a6d_10001\"));\n\t\tList<Document> documents = cloudStore.similaritySearch(\"南方电网\");\n\t\tSystem.out.println(documents.size());\n\t}\n\n}", "model_target_text": "\t\tCountDownLatch cdl = new CountDownLatch(1);\n\t\tresponse.subscribe(data -> {\n\t\t\tSystem.out.printf(\"%s\", data.getResult().getOutput().getContent());\n\t\t}, err -> {\n\t\t\tlogger.error(\"err: {}\", err.getMessage(), err);\n\t\t\tcdl.countDown();\n\t\t}, () -> {\n\t\t\tSystem.out.println();\n\t\t\tlogger.info(\"done\");\n\t\t\tcdl.countDown();\n\t\t});\n\n\t\tcdl.await();\n\t}", "region_before": "}\n\n\t@Test\n\tvoid callWithReferencedRagTest() throws IOException {\n\t\tDocumentRetriever retriever = new DashScopeDocumentRetriever(dashscopeApi,\n\t\t\t\tDashScopeDocumentRetrieverOptions.builder().withIndexName(\"spring-ai知识库\").build());\n\n\t\tChatClient chatClient = ChatClient.builder(dashscopeChatModel)\n\t\t\t.defaultAdvisors(new DashScopeDocumentRetrievalAdvisor(retriever,\n\t\t\t\t\tsystemResourceRef.getContentAsString(StandardCharsets.UTF_8), true))\n\t\t\t.build();\n\n\t\tChatResponse response = chatClient.prompt().user(\"如何快速开始百炼?\").call().chatResponse();", "jaccard_similarity": 0.037037037037037035, "semantic_cosine_similarity": 1.0000001192092896, "llm_judge": {"label": "BAD", "confidence": 5, "reasons": "The target snippet extends beyond the editable region end marker, including code that is outside the specified editable region, causing a mismatch with the region boundaries.", "flags": ["MISMATCHED_REGION"]}}
{"c